In [3]:
import json
import requests
from tinydb import TinyDB, Query
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

In [4]:
load_dotenv()
key = os.getenv("LANGSEARCH_API_KEY")
db = TinyDB("search_cache.json")

In [5]:
import requests
import json

url = "https://api.langsearch.com/v1/web-search"

payload = json.dumps({
  "query": "Tell me about the compant KnotAPI in detail",
  "freshness": "noLimit",
  "summary": True,
  "count": 10
})
headers = {
  'Authorization': f'Bearer {key}',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"code":200,"log_id":"994e693be65581e1","msg":null,"data":{"_type":"SearchResponse","queryContext":{"originalQuery":"Tell me about the compant KnotAPI in detail"},"webPages":{"webSearchUrl":"https://langsearch.com/search?q=Tell me about the compant KnotAPI in detail","totalEstimatedMatches":null,"value":[{"id":"https://api.langsearch.com/v1/#WebPages.1","name":"KnotAPI | Instant SKU-level data","url":"https://www.knotapi.com/txn-link/","displayUrl":"https://www.knotapi.com/txn-link/","snippet":"transaction link ™ \n sku - level data has arrived \n from tylenol to tide pods to transit , unlock item - level transaction insights from top merchants like walmart , amazon , and uber . powering perso...","summary":"transaction link ™ \n sku - level data has arrived \n from tylenol to tide pods to transit , unlock item - level transaction insights from top merchants like walmart , amazon , and uber . powering personalized offers , smarter analytics , and better customer experiences . \n truste

In [6]:
class Config:
    BASE_URL = "https://api.langsearch.com/v1/web-search"
    API_KEY = os.getenv("LANGSEARCH_API_KEY")
    DB_PATH = "search_cache.json"
    count = 10
    freshness = "noLimit"
    summary = True

In [7]:
import json
import requests
from tinydb import TinyDB, Query
from IPython.display import display, Markdown
from textwrap import shorten
from urllib.parse import urlparse

DEFAULT_BASE_URL = Config.BASE_URL
DEFAULT_DB_PATH  = Config.DB_PATH


def _safe(s: str | None, fallback: str = "") -> str:
    return (s or fallback).strip()


def _pretty_host(u: str | None) -> str:
    if not u:
        return ""
    try:
        host = urlparse(u).netloc
        return host.replace("www.", "")
    except Exception:
        return ""


class LangSearchClient:
    def __init__(self, api_key: str, db_path: str = DEFAULT_DB_PATH, endpoint: str | None = DEFAULT_BASE_URL):
        """
        Initialize client with API key, TinyDB path, and optional endpoint override.
        """
        if endpoint is None:
            raise ValueError("Missing BASE_URL. Pass endpoint=... or define Config.BASE_URL.")
        self.api_key = api_key
        self.db = TinyDB(db_path)
        self.endpoint = endpoint

    def search(self, query: str, count: int = 10, freshness: str = "noLimit", summary: bool = True):
        payload = {
            "query": query,
            "count": count,
            "freshness": freshness,
            "summary": summary,
        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        r = requests.post(self.endpoint, headers=headers, json=payload, timeout=45)
        if r.status_code != 200:
            raise RuntimeError(f"API error {r.status_code}: {r.text}")

        data = r.json()
        results = data.get("data", {}).get("webPages", {}).get("value", []) or []
        if not results:
            print("No results found.")
            return []

        entries = []
        for res in results:
            record = {
                "query": query,
                "name": _safe(res.get("name"), "Untitled Result"),
                "url": _safe(res.get("url")),
                "displayUrl": _safe(res.get("displayUrl"), _safe(res.get("url"))),
                "snippet": _safe(res.get("snippet")),
                "summary": _safe(res.get("summary")),
            }
            self.db.insert(record)
            entries.append(record)

        print(f"Stored {len(entries)} results for '{query}'")
        return entries

    # --- DROP-IN REPLACEMENT STARTS HERE ---
    def display_results(self, query: str, limit: int = 10, show_snippet: bool = True, show_summary: bool = True):
        """
        Display stored search results in clean, structured Markdown.
        - Renders once (no duplicate prints)
        - Handles empty fields gracefully
        - Truncates very long summaries
        """
        Q = Query()
        rows = self.db.search(Q.query == query)
        if not rows:
            print(f"No stored results found for '{query}'.")
            return

        # Stable ordering: by name then url; apply limit
        rows = sorted(rows, key=lambda r: (r.get("name") or "", r.get("url") or ""))[:limit]

        md = []
        md.append(f"# 🔍 Results for **{query}**")
        md.append("")
        md.append(f"*Showing {len(rows)} stored result(s).*")
        md.append("")

        for i, r in enumerate(rows, 1):
            title = _safe(r.get("name"), "Untitled Result")
            url = _safe(r.get("url"))
            durl = _safe(r.get("displayUrl"), url)
            host = _pretty_host(url)
            snippet = _safe(r.get("snippet"))
            summary = _safe(r.get("summary"))


            # Card header
            md.append(f"---")
            md.append(f"### {i}. [{title}]({url})")
            if host:
                md.append(f"*{host}*  ")
            else:
                md.append("")

            # Link row
            if url:
                md.append(f"**🌐 URL:** [{durl}]({url})")
            else:
                md.append("**🌐 URL:** _(missing)_")

            # Optional snippet
            if show_snippet and snippet:
                md.append("")
                md.append("**Snippet**")
                md.append(f"> {snippet}")

            # Optional summary
            if show_summary and summary:
                md.append("")
                md.append("**Summary**")
                md.append(f"> {summary}")

            md.append("")  # space after each card

        # Render once
        display(Markdown("\n".join(md)))
    # --- DROP-IN REPLACEMENT ENDS HERE ---

    def clear(self):
        self.db.truncate()
        print("🧹 Database cleared.")

    def list_queries(self):
        return sorted({r.get("query") for r in self.db.all() if r.get("query")})


In [10]:

client = LangSearchClient(api_key=Config.API_KEY, db_path=Config.DB_PATH)
results = client.search("Karan Pratap Singh System Design")
client.display_results("Karan Pratap Singh System Design")


Stored 10 results for 'Karan Pratap Singh System Design'


# 🔍 Results for **Karan Pratap Singh System Design**

*Showing 10 stored result(s).*

---
### 1. [Acing the System Design Interview - Zhiyong Tan](https://www.manning.com/books/acing-the-system-design-interview)
*manning.com*  
**🌐 URL:** [https://www.manning.com/books/acing-the-system-design-interview](https://www.manning.com/books/acing-the-system-design-interview)

**Snippet**
> acing the system design interview 
 zhiyong tan 
 foreword by anthony asta , michael d. elder 
 january 2024 
 isbn 9781633439108 
 472 pages 
 printed in black & white 
 available translations : kore...

**Summary**
> acing the system design interview 
 zhiyong tan 
 foreword by anthony asta , michael d. elder 
 january 2024 
 isbn 9781633439108 
 472 pages 
 printed in black & white 
 available translations : korean , russian , simplified chinese 
 general 
 ebook 
 $ 47 . 99 $ 31 . 19 
 you save $ 16 . 80 ( 35 % ) 
 the system design interview is one of the hardest challenges you’ll face in the software engineering hiring process . this practical book gives you the insights , the skills , and the hands - on practice you need to ace the toughest system design interview questions and land the job and salary you want . 
 in acing the system design interview you will master a structured and organized approach to present system design ideas like : 
 scaling applications to support heavy traffic 
 distributed transactions techniques to ensure data consistency 
 services for functional partitioning such as api gateway and service mesh 
 common api paradigms including rest , rpc , and graphql 
 caching strategies , including their tradeoffs 
 logging , monitoring , and alerting concepts that are critical in any system design 
 communication skills that demonstrate your engineering maturity 
 don’t be daunted by the complex , open - ended nature of system design interviews ! in this in - depth guide , author zhiyong tan shares what he ’s learned on both sides of the interview table . you’ll dive deep into the common technical topics that arise during interviews and learn how to apply them to mentally perfect different kinds of systems . 
 about the technology 
 the system design interview is daunting even for seasoned software engineers . fortunately , with a little careful prep work you can turn those open - ended questions and whiteboard sessions into your competitive advantage ! in this powerful book , zhiyong tan reveals practical interview techniques and insights about system design that have earned developers job offers from amazon , apple , bytedance , paypal , and uber . 
 about the book 
 acing the system design interview is a masterclass in how to confidently nail your next interview . following these easy - to - remember techniques , you’ll learn to quickly assess a question , identify an advantageous approach , and then communicate your ideas clearly to an interviewer . as you work through this book , you’ll gain not only the skills to successfully interview , but also to do the actual work of great system design . 
 what 's inside 
 insights on scaling , transactions , logging , and more 
 practice questions for core system design concepts 
 how to demonstrate your engineering maturity 
 great questions to ask your interviewer 
 about the reader 
 for software engineers , software architects , and engineering managers looking to advance their careers . 
 about the author 
 zhiyong tan is a manager at paypal . he has worked at uber , teradata , and at small startups . over the years , he has been in many system design interviews , on both sides of the table . 
 the technical editor on this book was mohit kumar .

---
### 2. [Brij kishore Pandey on LinkedIn: #systemdesign #coding #softwareengineering | 331 comments](http://www.linkedin.com/posts/brijpandeyji_systemdesign-coding-softwareengineering-activity-7081232542984511489-pJFv)
*linkedin.com*  
**🌐 URL:** [http://www.linkedin.com/posts/brijpandeyji_systemdesign-coding-softwareengineering-activity-7081232542984511489-pJFv](http://www.linkedin.com/posts/brijpandeyji_systemdesign-coding-softwareengineering-activity-7081232542984511489-pJFv)

**Snippet**
> brij kishore pandey ’s post 
 𝗡𝗲𝘁𝗳𝗹𝗶𝘅 '𝘀 𝗔𝗿𝗰𝗵𝗶𝘁𝗲𝗰𝘁𝘂𝗿𝗲 : 𝗔 𝗕𝗲𝗵𝗶𝗻𝗱 - 𝘁𝗵𝗲 - 𝗦𝗰𝗲𝗻𝗲𝘀 𝗟𝗼𝗼𝗸 netflix is one of the most popular streaming services in the world , and it...

**Summary**
> brij kishore pandey ’s post 
 𝗡𝗲𝘁𝗳𝗹𝗶𝘅 '𝘀 𝗔𝗿𝗰𝗵𝗶𝘁𝗲𝗰𝘁𝘂𝗿𝗲 : 𝗔 𝗕𝗲𝗵𝗶𝗻𝗱 - 𝘁𝗵𝗲 - 𝗦𝗰𝗲𝗻𝗲𝘀 𝗟𝗼𝗼𝗸 netflix is one of the most popular streaming services in the world , and its architecture is a big part of why it 's so successful . netflix uses a variety of technologies to deliver high - quality video to its users , and its architecture is constantly evolving to meet the needs of its growing customer base . 𝗛𝗲𝗿𝗲 𝗮𝗿𝗲 𝘀𝗼𝗺𝗲 𝗼𝗳 𝘁𝗵𝗲 𝗸𝗲𝘆 𝘁𝗲𝗰𝗵𝗻𝗼𝗹𝗼𝗴𝗶𝗲𝘀 𝘁𝗵𝗮𝘁 𝗡𝗲𝘁𝗳𝗹𝗶𝘅 𝘂𝘀𝗲𝘀 : 𝗠𝗼𝗯𝗶𝗹𝗲 𝗮𝗻𝗱 𝘄𝗲𝗯 : netflix has adopted swift and kotlin to build native mobile apps . for its web application , it uses react . 𝗙𝗿𝗼𝗻𝘁𝗲𝗻𝗱 / 𝘀𝗲𝗿𝘃𝗲𝗿 𝗰𝗼𝗺𝗺𝘂𝗻𝗶𝗰𝗮𝘁𝗶𝗼𝗻 : netflix uses graphql . 𝗕𝗮𝗰𝗸𝗲𝗻𝗱 𝘀𝗲𝗿𝘃𝗶𝗰𝗲𝘀 : netflix relies on zuul , eureka , the spring boot framework , and other technologies . 𝗗𝗮𝘁𝗮𝗯𝗮𝘀𝗲𝘀 : netflix utilizes ev cache , cassandra , cockroachdb , and other databases . 𝗠𝗲𝘀𝘀𝗮𝗴𝗶𝗻𝗴 / 𝘀𝘁𝗿𝗲𝗮𝗺𝗶𝗻𝗴 : netflix employs apache kafka and fink for messaging and streaming purposes . 𝗩𝗶𝗱𝗲𝗼 𝘀𝘁𝗼𝗿𝗮𝗴𝗲 : netflix uses s3 and open connect for video storage . 𝗗𝗮𝘁𝗮 𝗽𝗿𝗼𝗰𝗲𝘀𝘀𝗶𝗻𝗴 : netflix utilizes flink and spark for data processing , which is then visualized using tableau . redshift is used for processing structured data warehouse information . 𝗖𝗜 / 𝗖𝗗 : netflix employs various tools such as jira , confluence , pagerduty , jenkins , gradle , chaos monkey , spinnaker , altas , and more for ci/cd processes . netflix 's architecture is designed to be scalable , reliable , and secure . it 's also designed to be flexible , so that netflix can easily add new features and services . if you 're interested in learning more about netflix 's architecture , i recommend checking out the following resources : netflix tech blog : https : / / lnkd . in / d fpm 7 xxz netflix open source : https : / / netflix . github . io / netflix architecture : https : / / lnkd . in / du mxw hnd i hope this post gives you a better understanding of the architecture of netflix . if you have any questions , please feel free to leave a comment . what do you think about netflix 's architecture ? source - https : / / lnkd . in / dqr qq dbd 🌟 enjoying my content ? stay in the loop ! 🌟 - - - - - - - - - 1 ️⃣ follow me here : brij kishore pandey & 2️⃣ select " turn on notifications " by clicking the bell icon 🔔 on my profile . dive into a world of top - tier content and insights . connect with me on : 📱 telegram for real - time updates : https : / / t . me / code with brij 📸 instagram for engaging visuals : https : / / lnkd . in / dzj pve bk 🐦 twitter for quick thoughts and shares : https : / / lnkd . in / dkj yz 4 wk 
 # system design # coding # software engineering

---
### 3. [GitHub - chauhanrushang/system-design-primer: Learn how to design large-scale systems. Prep for the system design interview. Includes Anki flashcards.](https://github.com/chauhanrushang/system-design-primer)
*github.com*  
**🌐 URL:** [https://github.com/chauhanrushang/system-design-primer](https://github.com/chauhanrushang/system-design-primer)

**Snippet**
> chauhan rushang / 
 system - design - primer 
 public 
 chauhan rushang / system - design - primer 
 folders and files 
 name name last commit message last commit date 
 329 commits 
 .github 
 .githu...

**Summary**
> chauhan rushang / 
 system - design - primer 
 public 
 chauhan rushang / system - design - primer 
 folders and files 
 name name last commit message last commit date 
 329 commits 
 .github 
 .github 
 images 
 images 
 resources 
 resources 
 solutions 
 solutions 
 .gitattributes 
 .gitattributes 
 .gitignore 
 .gitignore 
 contributing.md 
 contributing.md 
 license.txt 
 license.txt 
 readme - ja . md 
 readme - ja . md 
 readme - zh - hans . md 
 readme - zh - hans . md 
 readme - zh - tw . md 
 readme - zh - tw . md 
 readme.md 
 readme.md 
 translations . md 
 translations . md 
 epub - metadata . yaml 
 epub - metadata . yaml 
 generate - epub . sh 
 generate - epub . sh 
 repository files navigation 
 the system design primer 
 motivation 
 learn how to design large - scale systems . 
 prep for the system design interview . 
 learn how to design large - scale systems 
 learning how to design scalable systems will help you become a better engineer . 
 system design is a broad topic . there is a vast amount of resources scattered throughout the web on system design principles . 
 this repo is an organized collection of resources to help you learn how to build systems at scale . 
 learn from the open source community 
 this is a continually updated , open source project . 
 contributions are welcome ! 
 prep for the system design interview 
 in addition to coding interviews , system design is a required component of the technical interview process at many tech companies . 
 practice common system design interview questions and compare your results with sample solutions : discussions , code , and diagrams . 
 additional topics for interview prep : 
 study guide 
 how to approach a system design interview question 
 system design interview questions , with solutions 
 object - oriented design interview questions , with solutions 
 additional system design interview questions 
 anki flashcards 
 the provided anki flashcard decks use spaced repetition to help you retain key system design concepts . 
 system design deck 
 system design exercises deck 
 object oriented design exercises deck 
 great for use while on - the - go . 
 coding resource : interactive coding challenges 
 looking for resources to help you prep for the coding interview ? 
 check out the sister repo interactive coding challenges , which contains an additional anki deck : 
 coding deck 
 contributing 
 learn from the community . 
 feel free to submit pull requests to help : 
 fix errors 
 improve sections 
 add new sections 
 translate 
 content that needs some polishing is placed under development . 
 review the contributing guidelines . 
 index of system design topics 
 summaries of various system design topics , including pros and cons . everything is a trade - off . 
 each section contains links to more in - depth resources . 
 system design topics : start here 
 step 1: review the scalability video lecture 
 step 2: review the scalability article 
 next steps 
 performance vs scalability 
 latency vs throughput 
 availability vs consistency 
 cap theorem 
 cp - consistency and partition tolerance 
 ap - availability and partition tolerance 
 consistency patterns 
 weak consistency 
 eventual consistency 
 strong consistency 
 availability patterns 
 fail - over 
 replication 
 availability in numbers 
 domain name system 
 content delivery network 
 push cdns 
 pull cdns 
 load balancer 
 active - passive 
 active - active 
 layer 4 load balancing 
 layer 7 load balancing 
 horizontal scaling 
 reverse proxy ( web server ) 
 load balancer vs reverse proxy 
 application layer 
 microservices 
 service discovery 
 database 
 relational database management system ( rdbms ) 
 master - slave replication 
 master - master replication 
 federation 
 sharding 
 denormalization 
 sql tuning 
 nosql 
 key - value store 
 document store 
 wide column store 
 graph database 
 sql or nosql 
 cache 
 client caching 
 cdn caching 
 web server caching 
 database caching 
 application caching 
 caching at the database query level 
 caching at the object level 
 when to update the cache 
 cache - aside 
 write - through 
 write - behind ( write - back ) 
 refresh - ahead 
 asynchronism 
 message queues 
 task queues 
 back pressure 
 communication 
 transmission control protocol ( tcp ) 
 user datagram protocol ( udp ) 
 remote procedure call ( rpc ) 
 representational state transfer ( rest ) 
 security 
 appendix 
 powers of two table 
 latency numbers every programmer should know 
 additional system design interview questions 
 real world architectures 
 company architectures 
 company engineering blogs 
 under development 
 credits 
 contact info 
 license 
 study guide 
 suggested topics to review based on your interview timeline ( short , medium , long ) . 
 q : for interviews , do i need to know everything here ? 
 a : no , you don't need to know everything here to prepare for the interview . 
 what you are asked in an interview depends on variables such as : 
 how much experience you have 
 what your technical background is 
 what positions you are interviewing for 
 which companies you are interviewing with 
 luck 
 more experienced candidates are generally expected to know more about system design . architects or team leads might be expected to know more than individual contributors . top tech companies are likely to have one or more design interview rounds . 
 start broad and go deeper in a few areas . it helps to know a little about various key system design topics . adjust the following guide based on your timeline , experience , what positions you are interviewing for , and which companies you are interviewing with . 
 short timeline - aim for breadth with system design topics . practice by solving some interview questions . 
 medium timeline - aim for breadth and some depth with system design topics . practice by solving many interview questions . 
 long timeline - aim for breadth and more depth with system design topics . practice by solving most interview questions . 
 short medium long 
 read through the system design topics to get a broad understanding of how systems work 👍 👍 👍 
 read through a few articles in the company engineering blogs for the companies you are interviewing with 👍 👍 👍 
 read through a few real world architectures 👍 👍 👍 
 review how to approach a system design interview question 👍 👍 👍 
 work through system design interview questions with solutions some many most 
 work through object - oriented design interview questions with solutions some many most 
 review additional system design interview questions some many most 
 how to approach a system design interview question 
 how to tackle a system design interview question . 
 the system design interview is an open - ended conversation . you are expected to lead it . 
 you can use the following steps to guide the discussion . to help solidify this process , work through the system design interview questions with solutions section using the following steps . 
 step 1: outline use cases , constraints , and assumptions 
 gather requirements and scope the problem . ask questions to clarify use cases and constraints . discuss assumptions . 
 who is going to use it ? 
 how are they going to use it ? 
 how many users are there ? 
 what does the system do ? 
 what are the inputs and outputs of the system ? 
 how much data do we expect to handle ? 
 how many requests per second do we expect ? 
 what is the expected read to write ratio ? 
 step 2: create a high level design 
 outline a high level design with all important components . 
 sketch the main components and connections 
 justify your ideas 
 step 3: design core components 
 dive into details for each core component . for example , if you were asked to design a url shortening service , discuss : 
 generating and storing a hash of the full url 
 md5 and base62 
 hash collisions 
 sql or nosql 
 database schema 
 translating a hashed url to the full url 
 database lookup 
 api and object - oriented design 
 step 4: scale the design 
 identify and address bottlenecks , given the constraints . for example , do you need the following to address scalability issues ? 
 load balancer 
 horizontal scaling 
 caching 
 database sharding 
 discuss potential solutions and trade - offs . everything is a trade - off . address bottlenecks using principles of scalable system design . 
 back - of - the - envelope calculations 
 you might be asked to do some estimates by hand . refer to the appendix for the following resources : 
 use back of the envelope calculations 
 powers of two table 
 latency numbers every programmer should know 
 source ( s ) and further reading 
 system design interview questions with solutions 
 common system design interview questions with sample discussions , code , and diagrams . 
 solutions linked to content in the solutions / folder . 
 question 
 design pastebin.com ( or bit . ly ) solution 
 design the twitter timeline and search ( or facebook feed and search ) solution 
 design a web crawler solution 
 design mint.com solution 
 design the data structures for a social network solution 
 design a key - value store for a search engine solution 
 design amazon 's sales ranking by category feature solution 
 design a system that scales to millions of users on aws solution 
 add a system design question contribute 
 design pastebin.com ( or bit . ly ) 
 design the twitter timeline and search ( or facebook feed and search ) 
 design a web crawler 
 design mint.com 
 design the data structures for a social network 
 design a key - value store for a search engine 
 design amazon 's sales ranking by category feature 
 design a system that scales to millions of users on aws 
 object - oriented design interview questions with solutions 
 common object - oriented design interview questions with sample discussions , code , and diagrams . 
 solutions linked to content in the solutions / folder . 
 note : this section is under development 
 question 
 design a hash map solution 
 design a least recently used cache solution 
 design a call center solution 
 design a deck of cards solution 
 design a parking lot solution 
 design a chat server solution 
 design a circular array contribute 
 add an object - oriented design question contribute 
 system design topics : start here 
 new to system design ? 
 first , you'll need a basic understanding of common principles , learning about what they are , how they are used , and their pros and cons . 
 step 1: review the scalability video lecture 
 scalability lecture at harvard 
 topics covered : 
 vertical scaling 
 horizontal scaling 
 caching 
 load balancing 
 database replication 
 database partitioning 
 step 2: review the scalability article 
 scalability 
 topics covered : 
 clones 
 databases 
 caches 
 asynchronism 
 next steps 
 next , we 'll look at high - level trade - offs : 
 performance vs scalability 
 latency vs throughput 
 availability vs consistency 
 keep in mind that everything is a trade-off . 
 then we 'll dive into more specific topics such as dns , cdns , and load balancers . 
 performance vs scalability 
 a service is scalable if it results in increased performance in a manner proportional to resources added . generally , increasing performance means serving more units of work , but it can also be to handle larger units of work , such as when datasets grow . 1 
 another way to look at performance vs scalability : 
 if you have a performance problem , your system is slow for a single user . 
 if you have a scalability problem , your system is fast for a single user but slow under heavy load . 
 latency vs throughput 
 latency is the time to perform some action or to produce some result . 
 throughput is the number of such actions or results per unit of time . 
 generally , you should aim for maximal throughput with acceptable latency . 
 availability vs consistency 
 cap theorem 
 source : cap theorem revisited 
 in a distributed computer system , you can only support two of the following guarantees : 
 consistency - every read receives the most recent write or an error 
 availability - every request receives a response , without guarantee that it contains the most recent version of the information 
 partition tolerance - the system continues to operate despite arbitrary partitioning due to network failures 
 networks aren't reliable , so you'll need to support partition tolerance . you'll need to make a software tradeoff between consistency and availability . 
 cp - consistency and partition tolerance 
 waiting for a response from the partitioned node might result in a timeout error . cp is a good choice if your business needs require atomic reads and writes . 
 ap - availability and partition tolerance 
 responses return the most readily available version of the data available on any node , which might not be the latest . writes might take some time to propagate when the partition is resolved . 
 ap is a good choice if the business needs to allow for eventual consistency or when the system needs to continue working despite external errors . 
 consistency patterns 
 with multiple copies of the same data , we are faced with options on how to synchronize them so clients have a consistent view of the data . recall the definition of consistency from the cap theorem - every read receives the most recent write or an error . 
 weak consistency 
 after a write , reads may or may not see it . a best effort approach is taken . 
 this approach is seen in systems such as memcached . weak consistency works well in real time use cases such as voip , video chat , and realtime multiplayer games . for example , if you are on a phone call and lose reception for a few seconds , when you regain connection you do not hear what was spoken during connection loss . 
 eventual consistency 
 after a write , reads will eventually see it ( typically within milliseconds ) . data is replicated asynchronously . 
 this approach is seen in systems such as dns and email . eventual consistency works well in highly available systems . 
 strong consistency 
 after a write , reads will see it . data is replicated synchronously . 
 this approach is seen in file systems and rdbms es . strong consistency works well in systems that need transactions . 
 availability patterns 
 there are two complementary patterns to support high availability : fail - over and replication . 
 fail - over 
 active - passive 
 with active - passive fail - over , heartbeats are sent between the active and the passive server on standby . if the heartbeat is interrupted , the passive server takes over the active 's ip address and resumes service . 
 the length of downtime is determined by whether the passive server is already running in ' hot ' standby or whether it needs to start up from ' cold ' standby . only the active server handles traffic . 
 active - passive failover can also be referred to as master - slave failover . 
 active - active 
 in active - active , both servers are managing traffic , spreading the load between them . 
 if the servers are public - facing , the dns would need to know about the public ips of both servers . if the servers are internal - facing , application logic would need to know about both servers . 
 active - active failover can also be referred to as master - master failover . 
 disadvantage ( s ) : failover 
 fail - over adds more hardware and additional complexity . 
 there is a potential for loss of data if the active system fails before any newly written data can be replicated to the passive . 
 replication 
 master - slave and master - master 
 this topic is further discussed in the database section : 
 master - slave replication 
 master - master replication 
 availability in numbers 
 availability is often quantified by uptime ( or downtime ) as a percentage of time the service is available . availability is generally measured in number of 9 s - - a service with 99 . 99 % availability is described as having four 9 s . 
 99 . 9 % availability - three 9s 
 duration acceptable downtime 
 downtime per year 8h 45min 57s 
 downtime per month 43m 49 . 7 s 
 downtime per week 10m 4 . 8 s 
 downtime per day 1m 26 . 4 s 
 99 . 99 % availability - four 9s 
 duration acceptable downtime 
 downtime per year 52min 35 . 7 s 
 downtime per month 4m 23s 
 downtime per week 1m 5s 
 downtime per day 8 . 6 s 
 availability in parallel vs in sequence 
 if a service consists of multiple components prone to failure , the service 's overall availability depends on whether the components are in sequence or in parallel . 
 in sequence 
 overall availability decreases when two components with availability < 100 % are in sequence : 
 availability ( total ) = availability ( foo ) * availability ( bar ) 
 if both foo and bar each had 99 . 9 % availability , their total availability in sequence would be 99 . 8 % . 
 in parallel 
 overall availability increases when two components with availability < 100 % are in parallel : 
 availability ( total ) = 1 - ( 1 - availability ( foo ) ) * ( 1 - availability ( bar ) ) 
 if both foo and bar each had 99 . 9 % availability , their total availability in parallel would be 99 . 9999 % . 
 domain name system 
 source : dns security presentation 
 a domain name system ( dns ) translates a domain name such as www.example.com to an ip address . 
 dns is hierarchical , with a few authoritative servers at the top level . your router or isp provides information about which dns server ( s ) to contact when doing a lookup . lower level dns servers cache mappings , which could become stale due to dns propagation delays . dns results can also be cached by your browser or os for a certain period of time , determined by the time to live ( ttl ) . 
 ns record ( name server ) - specifies the dns servers for your domain / subdomain . 
 mx record ( mail exchange ) - specifies the mail servers for accepting messages . 
 a record ( address ) - points a name to an ip address . 
 cname ( canonical ) - points a name to another name or cname ( example . com to www . example . com ) or to an a record . 
 services such as cloudflare and route 53 provide managed dns services . some dns services can route traffic through various methods : 
 weighted round robin 
 prevent traffic from going to servers under maintenance 
 balance between varying cluster sizes 
 a/b testing 
 latency - based 
 geolocation - based 
 disadvantage ( s ) : dns 
 accessing a dns server introduces a slight delay , although mitigated by caching described above . 
 dns server management could be complex and is generally managed by governments , isps , and large companies . 
 dns services have recently come under ddos attack , preventing users from accessing websites such as twitter without knowing twitter 's ip address ( es ) . 
 content delivery network 
 source : why use a cdn 
 a content delivery network ( cdn ) is a globally distributed network of proxy servers , serving content from locations closer to the user . generally , static files such as html / css / js , photos , and videos are served from cdn , although some cdns such as amazon 's cloudfront support dynamic content . the site 's dns resolution will tell clients which server to contact . 
 serving content from cdns can significantly improve performance in two ways : 
 users receive content from data centers close to them 
 your servers do not have to serve requests that the cdn fulfills 
 push cdns 
 push cdns receive new content whenever changes occur on your server . you take full responsibility for providing content , uploading directly to the cdn and rewriting urls to point to the cdn . you can configure when content expires and when it is updated . content is uploaded only when it is new or changed , minimizing traffic , but maximizing storage . 
 sites with a small amount of traffic or sites with content that isn't often updated work well with push cdns . content is placed on the cdns once , instead of being re - pulled at regular intervals . 
 pull cdns 
 pull cdns grab new content from your server when the first user requests the content . you leave the content on your server and rewrite urls to point to the cdn . this results in a slower request until the content is cached on the cdn . 
 a time - to - live ( ttl ) determines how long content is cached . pull cdns minimize storage space on the cdn , but can create redundant traffic if files expire and are pulled before they have actually changed . 
 sites with heavy traffic work well with pull cdns , as traffic is spread out more evenly with only recently - requested content remaining on the cdn . 
 disadvantage ( s ) : cdn 
 cdn costs could be significant depending on traffic , although this should be weighed with additional costs you would incur not using a cdn . 
 content might be stale if it is updated before the ttl expires it . 
 cdns require changing urls for static content to point to the cdn . 
 load balancer 
 source : scalable system design patterns 
 load balancers distribute incoming client requests to computing resources such as application servers and databases . in each case , the load balancer returns the response from the computing resource to the appropriate client . load balancers are effective at : 
 preventing requests from going to unhealthy servers 
 preventing overloading resources 
 helping to eliminate a single point of failure 
 load balancers can be implemented with hardware ( expensive ) or with software such as haproxy . 
 additional benefits include : 
 ssl termination - decrypt incoming requests and encrypt server responses so backend servers do not have to perform these potentially expensive operations 
 removes the need to install x.509 certificates on each server 
 session persistence - issue cookies and route a specific client 's requests to same instance if the web apps do not keep track of sessions 
 to protect against failures , it 's common to set up multiple load balancers , either in active - passive or active - active mode . 
 load balancers can route traffic based on various metrics , including : 
 random 
 least loaded 
 session / cookies 
 round robin or weighted round robin 
 layer 4 
 layer 7 
 layer 4 load balancing 
 layer 4 load balancers look at info at the transport layer to decide how to distribute requests . generally , this involves the source , destination ip addresses , and ports in the header , but not the contents of the packet . layer 4 load balancers forward network packets to and from the upstream server , performing network address translation ( nat ) . 
 layer 7 load balancing 
 layer 7 load balancers look at the application layer to decide how to distribute requests . this can involve contents of the header , message , and cookies . layer 7 load balancers terminate network traffic , reads the message , makes a load - balancing decision , then opens a connection to the selected server . for example , a layer 7 load balancer can direct video traffic to servers that host videos while directing more sensitive user billing traffic to security - hardened servers . 
 at the cost of flexibility , layer 4 load balancing requires less time and computing resources than layer 7 , although the performance impact can be minimal on modern commodity hardware . 
 horizontal scaling 
 load balancers can also help with horizontal scaling , improving performance and availability . scaling out using commodity machines is more cost efficient and results in higher availability than scaling up a single server on more expensive hardware , called vertical scaling . it is also easier to hire for talent working on commodity hardware than it is for specialized enterprise systems . 
 disadvantage ( s ) : horizontal scaling 
 scaling horizontally introduces complexity and involves cloning servers 
 servers should be stateless : they should not contain any user - related data like sessions or profile pictures 
 sessions can be stored in a centralized data store such as a database ( sql , nosql ) or a persistent cache ( redis , memcached ) 
 downstream servers such as caches and databases need to handle more simultaneous connections as upstream servers scale out 
 disadvantage ( s ) : load balancer 
 the load balancer can become a performance bottleneck if it does not have enough resources or if it is not configured properly . 
 introducing a load balancer to help eliminate a single point of failure results in increased complexity . 
 a single load balancer is a single point of failure , configuring multiple load balancers further increases complexity . 
 reverse proxy ( web server ) 
 source : wikipedia 
 a reverse proxy is a web server that centralizes internal services and provides unified interfaces to the public . requests from clients are forwarded to a server that can fulfill it before the reverse proxy returns the server 's response to the client . 
 additional benefits include : 
 increased security - hide information about backend servers , blacklist ips , limit number of connections per client 
 increased scalability and flexibility - clients only see the reverse proxy 's ip , allowing you to scale servers or change their configuration 
 ssl termination - decrypt incoming requests and encrypt server responses so backend servers do not have to perform these potentially expensive operations 
 removes the need to install x.509 certificates on each server 
 compression - compress server responses 
 caching - return the response for cached requests 
 static content - serve static content directly 
 html / css / js 
 photos 
 videos 
 etc 
 load balancer vs reverse proxy 
 deploying a load balancer is useful when you have multiple servers . often , load balancers route traffic to a set of servers serving the same function . 
 reverse proxies can be useful even with just one web server or application server , opening up the benefits described in the previous section . 
 solutions such as nginx and haproxy can support both layer 7 reverse proxying and load balancing . 
 disadvantage ( s ) : reverse proxy 
 introducing a reverse proxy results in increased complexity . 
 a single reverse proxy is a single point of failure , configuring multiple reverse proxies ( ie a failover ) further increases complexity . 
 application layer 
 source : intro to architecting systems for scale 
 separating out the web layer from the application layer ( also known as platform layer ) allows you to scale and configure both layers independently . adding a new api results in adding application servers without necessarily adding additional web servers . the single responsibility principle advocates for small and autonomous services that work together . small teams with small services can plan more aggressively for rapid growth . 
 workers in the application layer also help enable asynchronism . 
 microservices 
 related to this discussion are microservices , which can be described as a suite of independently deployable , small , modular services . each service runs a unique process and communicates through a well - defined , lightweight mechanism to serve a business goal . 1 
 pinterest , for example , could have the following microservices : user profile , follower , feed , search , photo upload , etc. 
 service discovery 
 systems such as consul , etcd , and zookeeper can help services find each other by keeping track of registered names , addresses , and ports . health checks help verify service integrity and are often done using an http endpoint . both consul and etcd have a built in key - value store that can be useful for storing config values and other shared data . 
 disadvantage ( s ) : application layer 
 adding an application layer with loosely coupled services requires a different approach from an architectural , operations , and process viewpoint ( vs a monolithic system ) . 
 microservices can add complexity in terms of deployments and operations . 
 database 
 source : scaling up to your first 10 million users 
 relational database management system ( rdbms ) 
 a relational database like sql is a collection of data items organized in tables . 
 acid is a set of properties of relational database transactions . 
 atomicity - each transaction is all or nothing 
 consistency - any transaction will bring the database from one valid state to another 
 isolation - executing transactions concurrently has the same results as if the transactions were executed serially 
 durability - once a transaction has been committed , it will remain so 
 there are many techniques to scale a relational database : master - slave replication , master - master replication , federation , sharding , denormalization , and sql tuning . 
 master - slave replication 
 the master serves reads and writes , replicating writes to one or more slaves , which serve only reads . slaves can also replicate to additional slaves in a tree - like fashion . if the master goes offline , the system can continue to operate in read - only mode until a slave is promoted to a master or a new master is provisioned . 
 source : scalability , availability , stability , patterns 
 disadvantage ( s ) : master - slave replication 
 additional logic is needed to promote a slave to a master . 
 see disadvantage ( s ) : replication for points related to both master - slave and master - master . 
 master - master replication 
 both masters serve reads and writes and coordinate with each other on writes . if either master goes down , the system can continue to operate with both reads and writes . 
 source : scalability , availability , stability , patterns 
 disadvantage ( s ) : master - master replication 
 you'll need a load balancer or you'll need to make changes to your application logic to determine where to write . 
 most master - master systems are either loosely consistent ( violating acid ) or have increased write latency due to synchronization . 
 conflict resolution comes more into play as more write nodes are added and as latency increases . 
 see disadvantage ( s ) : replication for points related to both master - slave and master - master . 
 disadvantage ( s ) : replication 
 there is a potential for loss of data if the master fails before any newly written data can be replicated to other nodes . 
 writes are replayed to the read replicas . if there are a lot of writes , the read replicas can get bogged down with replaying writes and can't do as many reads . 
 the more read slaves , the more you have to replicate , which leads to greater replication lag . 
 on some systems , writing to the master can spawn multiple threads to write in parallel , whereas read replicas only support writing sequentially with a single thread . 
 replication adds more hardware and additional complexity . 
 federation 
 source : scaling up to your first 10 million users 
 federation ( or functional partitioning ) splits up databases by function . for example , instead of a single , monolithic database , you could have three databases : forums , users , and products , resulting in less read and write traffic to each database and therefore less replication lag . smaller databases result in more data that can fit in memory , which in turn results in more cache hits due to improved cache locality . with no single central master serializing writes you can write in parallel , increasing throughput . 
 disadvantage ( s ) : federation 
 federation is not effective if your schema requires huge functions or tables . 
 you'll need to update your application logic to determine which database to read and write . 
 joining data from two databases is more complex with a server link . 
 federation adds more hardware and additional complexity . 
 sharding 
 source : scalability , availability , stability , patterns 
 sharding distributes data across different databases such that each database can only manage a subset of the data . taking a users database as an example , as the number of users increases , more shards are added to the cluster . 
 similar to the advantages of federation , sharding results in less read and write traffic , less replication , and more cache hits . index size is also reduced , which generally improves performance with faster queries . if one shard goes down , the other shards are still operational , although you'll want to add some form of replication to avoid data loss . like federation , there is no single central master serializing writes , allowing you to write in parallel with increased throughput . 
 common ways to shard a table of users is either through the user 's last name initial or the user 's geographic location . 
 disadvantage ( s ) : sharding 
 you'll need to update your application logic to work with shards , which could result in complex sql queries . 
 data distribution can become lopsided in a shard . for example , a set of power users on a shard could result in increased load to that shard compared to others . 
 rebalancing adds additional complexity . a sharding function based on consistent hashing can reduce the amount of transferred data . 
 joining data from multiple shards is more complex . 
 sharding adds more hardware and additional complexity . 
 denormalization 
 denormalization attempts to improve read performance at the expense of some write performance . redundant copies of the data are written in multiple tables to avoid expensive joins . some rdbms such as postgresql and oracle support materialized views which handle the work of storing redundant information and keeping redundant copies consistent . 
 once data becomes distributed with techniques such as federation and sharding , managing joins across data centers further increases complexity . denormalization might circumvent the need for such complex joins . 
 in most systems , reads can heavily outnumber writes 100:1 or even 1000 : 1 . a read resulting in a complex database join can be very expensive , spending a significant amount of time on disk operations . 
 disadvantage ( s ) : denormalization 
 data is duplicated . 
 constraints can help redundant copies of information stay in sync , which increases complexity of the database design . 
 a denormalized database under heavy write load might perform worse than its normalized counterpart . 
 sql tuning 
 sql tuning is a broad topic and many books have been written as reference . 
 it 's important to benchmark and profile to simulate and uncover bottlenecks . 
 benchmark - simulate high - load situations with tools such as ab . 
 profile - enable tools such as the slow query log to help track performance issues . 
 benchmarking and profiling might point you to the following optimizations . 
 tighten up the schema 
 mysql dumps to disk in contiguous blocks for fast access . 
 use char instead of varchar for fixed - length fields . 
 char effectively allows for fast , random access , whereas with varchar , you must find the end of a string before moving onto the next one . 
 use text for large blocks of text such as blog posts . text also allows for boolean searches . using a text field results in storing a pointer on disk that is used to locate the text block . 
 use int for larger numbers up to 2 ^ 32 or 4 billion . 
 use decimal for currency to avoid floating point representation errors . 
 avoid storing large blobs , store the location of where to get the object instead . 
 varchar ( 255 ) is the largest number of characters that can be counted in an 8 bit number , often maximizing the use of a byte in some rdbms . 
 set the not null constraint where applicable to improve search performance . 
 indices are usually represented as self - balancing b - tree that keeps data sorted and allows searches , sequential access , insertions , and deletions in logarithmic time . 
 placing an index can keep the data in memory , requiring more space . 
 writes could also be slower since the index also needs to be updated . 
 when loading large amounts of data , it might be faster to disable indices , load the data , then rebuild the indices . 
 avoid expensive joins 
 denormalize where performance demands it . 
 partition tables 
 break up a table by putting hot spots in a separate table to help keep it in memory . 
 tune the query cache 
 in some cases , the query cache could lead to performance issues . 
 nosql 
 nosql is a collection of data items represented in a key - value store , document store , wide column store , or a graph database . data is denormalized , and joins are generally done in the application code . most nosql stores lack true acid transactions and favor eventual consistency . 
 base is often used to describe the properties of nosql databases . in comparison with the cap theorem , base chooses availability over consistency . 
 basically available - the system guarantees availability . 
 soft state - the state of the system may change over time , even without input . 
 eventual consistency - the system will become consistent over a period of time , given that the system doesn't receive input during that period . 
 in addition to choosing between sql or nosql , it is helpful to understand which type of nosql database best fits your use case ( s ) . we 'll review key - value stores , document stores , wide column stores , and graph databases in the next section . 
 key - value store 
 abstraction : hash table 
 a key - value store generally allows for o ( 1 ) reads and writes and is often backed by memory or ssd . data stores can maintain keys in lexicographic order , allowing efficient retrieval of key ranges . key - value stores can allow for storing of metadata with a value . 
 key - value stores provide high performance and are often used for simple data models or for rapidly - changing data , such as an in - memory cache layer . since they offer only a limited set of operations , complexity is shifted to the application layer if additional operations are needed . 
 a key - value store is the basis for more complex systems such as a document store , and in some cases , a graph database . 
 key - value database 
 disadvantages of key - value stores 
 redis architecture 
 memcached architecture 
 document store 
 abstraction : key - value store with documents stored as values 
 a document store is centered around documents ( xml , json , binary , etc ) , where a document stores all information for a given object . document stores provide apis or a query language to query based on the internal structure of the document itself . note , many key - value stores include features for working with a value 's metadata , blurring the lines between these two storage types . 
 based on the underlying implementation , documents are organized by collections , tags , metadata , or directories . although documents can be organized or grouped together , documents may have fields that are completely different from each other . 
 some document stores like mongodb and couchdb also provide a sql - like language to perform complex queries . dynamodb supports both key - values and documents . 
 document stores provide high flexibility and are often used for working with occasionally changing data . 
 mongodb architecture 
 couchdb architecture 
 elasticsearch architecture 
 wide column store 
 source : sql & nosql , a brief history 
 abstraction : nested map column family < row key , columns < col key , value , timestamp > > 
 a wide column store 's basic unit of data is a column ( name / value pair ) . a column can be grouped in column families ( analogous to a sql table ) . super column families further group column families . you can access each column independently with a row key , and columns with the same row key form a row . each value contains a timestamp for versioning and for conflict resolution . 
 google introduced bigtable as the first wide column store , which influenced the open - source hbase often - used in the hadoop ecosystem , and cassandra from facebook . stores such as big table , hbase , and cassandra maintain keys in lexicographic order , allowing efficient retrieval of selective key ranges . 
 wide column stores offer high availability and high scalability . they are often used for very large data sets . 
 bigtable architecture 
 hbase architecture 
 cassandra architecture 
 graph database 
 source : graph database 
 abstraction : graph 
 in a graph database , each node is a record and each arc is a relationship between two nodes . graph databases are optimized to represent complex relationships with many foreign keys or many - to - many relationships . 
 graphs databases offer high performance for data models with complex relationships , such as a social network . they are relatively new and are not yet widely - used ; it might be more difficult to find development tools and resources . many graphs can only be accessed with rest apis . 
 graph database 
 neo4j 
 flock db 
 sql or nosql 
 source : transitioning from rdbms to nosql 
 reasons for sql : 
 structured data 
 strict schema 
 relational data 
 need for complex joins 
 transactions 
 clear patterns for scaling 
 more established : developers , community , code , tools , etc 
 lookups by index are very fast 
 reasons for nosql : 
 semi - structured data 
 dynamic or flexible schema 
 non - relational data 
 no need for complex joins 
 store many tb ( or pb ) of data 
 very data intensive workload 
 very high throughput for iops 
 sample data well - suited for nosql : 
 rapid ingest of clickstream and log data 
 leaderboard or scoring data 
 temporary data , such as a shopping cart 
 frequently accessed ( ' hot ' ) tables 
 metadata / lookup tables 
 cache 
 source : scalable system design patterns 
 caching improves page load times and can reduce the load on your servers and databases . in this model , the dispatcher will first lookup if the request has been made before and try to find the previous result to return , in order to save the actual execution . 
 databases often benefit from a uniform distribution of reads and writes across its partitions . popular items can skew the distribution , causing bottlenecks . putting a cache in front of a database can help absorb uneven loads and spikes in traffic . 
 client caching 
 caches can be located on the client side ( os or browser ) , server side , or in a distinct cache layer . 
 cdn caching 
 cdns are considered a type of cache . 
 web server caching 
 reverse proxies and caches such as varnish can serve static and dynamic content directly . web servers can also cache requests , returning responses without having to contact application servers . 
 database caching 
 your database usually includes some level of caching in a default configuration , optimized for a generic use case . tweaking these settings for specific usage patterns can further boost performance . 
 application caching 
 in - memory caches such as memcached and redis are key - value stores between your application and your data storage . since the data is held in ram , it is much faster than typical databases where data is stored on disk . ram is more limited than disk , so cache invalidation algorithms such as least recently used ( lru ) can help invalidate ' cold ' entries and keep ' hot ' data in ram . 
 redis has the following additional features : 
 persistence option 
 built - in data structures such as sorted sets and lists 
 there are multiple levels you can cache that fall into two general categories : database queries and objects : 
 row level 
 query - level 
 fully - formed serializable objects 
 fully - rendered html 
 generally , you should try to avoid file - based caching , as it makes cloning and auto - scaling more difficult . 
 caching at the database query level 
 whenever you query the database , hash the query as a key and store the result to the cache . this approach suffers from expiration issues : 
 hard to delete a cached result with complex queries 
 if one piece of data changes such as a table cell , you need to delete all cached queries that might include the changed cell 
 caching at the object level 
 see your data as an object , similar to what you do with your application code . have your application assemble the dataset from the database into a class instance or a data structure ( s ) : 
 remove the object from cache if its underlying data has changed 
 allows for asynchronous processing : workers assemble objects by consuming the latest cached object 
 suggestions of what to cache : 
 user sessions 
 fully rendered web pages 
 activity streams 
 user graph data 
 when to update the cache 
 since you can only store a limited amount of data in cache , you'll need to determine which cache update strategy works best for your use case . 
 cache - aside 
 source : from cache to in - memory data grid 
 the application is responsible for reading and writing from storage . the cache does not interact with storage directly . the application does the following : 
 look for entry in cache , resulting in a cache miss 
 load entry from the database 
 add entry to cache 
 return entry 
 def get _ user ( self , user _ id ) : 
 user = cache . get ( " user . { 0 } " , user _ id ) 
 if user is none : 
 user = db . query ( " select * from users where user _ id = { 0 } " , user _ id ) 
 if user is not none : 
 key = " user . { 0 } " . format ( user _ id ) 
 cache . set ( key , json . dumps ( user ) ) 
 return user 
 memcached is generally used in this manner . 
 subsequent reads of data added to cache are fast . cache - aside is also referred to as lazy loading . only requested data is cached , which avoids filling up the cache with data that isn't requested . 
 disadvantage ( s ) : cache - aside 
 each cache miss results in three trips , which can cause a noticeable delay . 
 data can become stale if it is updated in the database . this issue is mitigated by setting a time - to - live ( ttl ) which forces an update of the cache entry , or by using write - through . 
 when a node fails , it is replaced by a new , empty node , increasing latency . 
 write - through 
 source : scalability , availability , stability , patterns 
 the application uses the cache as the main data store , reading and writing data to it , while the cache is responsible for reading and writing to the database : 
 application adds / updates entry in cache 
 cache synchronously writes entry to data store 
 return 
 application code : 
 set _ user ( 12345 , { " foo " : " bar " } ) 
 cache code : 
 def set _ user ( user _ id , values ) : 
 user = db . query ( " update users where id = { 0 } " , user _ id , values ) 
 cache . set ( user _ id , user ) 
 write - through is a slow overall operation due to the write operation , but subsequent reads of just written data are fast . users are generally more tolerant of latency when updating data than reading data . data in the cache is not stale . 
 disadvantage ( s ) : write through 
 when a new node is created due to failure or scaling , the new node will not cache entries until the entry is updated in the database . cache - aside in conjunction with write through can mitigate this issue . 
 most data written might never be read , which can be minimized with a ttl . 
 write - behind ( write - back ) 
 source : scalability , availability , stability , patterns 
 in write - behind , the application does the following : 
 add / update entry in cache 
 asynchronously write entry to the data store , improving write performance 
 disadvantage ( s ) : write - behind 
 there could be data loss if the cache goes down prior to its contents hitting the data store . 
 it is more complex to implement write - behind than it is to implement cache - aside or write - through . 
 refresh - ahead 
 source : from cache to in - memory data grid 
 you can configure the cache to automatically refresh any recently accessed cache entry prior to its expiration . 
 refresh - ahead can result in reduced latency vs read - through if the cache can accurately predict which items are likely to be needed in the future . 
 disadvantage ( s ) : refresh - ahead 
 not accurately predicting which items are likely to be needed in the future can result in reduced performance than without refresh - ahead . 
 disadvantage ( s ) : cache 
 need to maintain consistency between caches and the source of truth such as the database through cache in validation . 
 cache invalidation is a difficult problem , there is additional complexity associated with when to update the cache . 
 need to make application changes such as adding redis or memcached . 
 source ( s ) and further reading 
 from cache to in - memory data grid 
 scalable system design patterns 
 introduction to architecting systems for scale 
 scalability , availability , stability , patterns 
 scalability 
 aws elasticache strategies 
 wikipedia 
 asynchronism 
 source : intro to architecting systems for scale 
 asynchronous workflows help reduce request times for expensive operations that would otherwise be performed in - line . they can also help by doing time - consuming work in advance , such as periodic aggregation of data . 
 message queues 
 message queues receive , hold , and deliver messages . if an operation is too slow to perform inline , you can use a message queue with the following workflow : 
 an application publishes a job to the queue , then notifies the user of job status 
 a worker picks up the job from the queue , processes it , then signals the job is complete 
 the user is not blocked and the job is processed in the background . during this time , the client might optionally do a small amount of processing to make it seem like the task has completed . for example , if posting a tweet , the tweet could be instantly posted to your timeline , but it could take some time before your tweet is actually delivered to all of your followers . 
 redis is useful as a simple message broker but messages can be lost . 
 rabbitmq is popular but requires you to adapt to the ' amqp ' protocol and manage your own nodes . 
 amazon sqs is hosted but can have high latency and has the possibility of messages being delivered twice . 
 task queues 
 tasks queues receive tasks and their related data , runs them , then delivers their results . they can support scheduling and can be used to run computationally - intensive jobs in the background . 
 celery has support for scheduling and primarily has python support . 
 back pressure 
 if queues start to grow significantly , the queue size can become larger than memory , resulting in cache misses , disk reads , and even slower performance . back pressure can help by limiting the queue size , thereby maintaining a high throughput rate and good response times for jobs already in the queue . once the queue fills up , clients get a server busy or http 503 status code to try again later . clients can retry the request at a later time , perhaps with exponential backoff . 
 disadvantage ( s ) : asynchronism 
 use cases such as inexpensive calculations and realtime workflows might be better suited for synchronous operations , as introducing queues can add delays and complexity . 
 source ( s ) and further reading 
 it 's all a numbers game 
 applying back pressure when overloaded 
 little 's law 
 what is the difference between a message queue and a task queue ? 
 communication 
 source : osi 7 layer model 
 hypertext transfer protocol ( http ) 
 http is a method for encoding and transporting data between a client and a server . it is a request / response protocol : clients issue requests and servers issue responses with relevant content and completion status info about the request . http is self - contained , allowing requests and responses to flow through many intermediate routers and servers that perform load balancing , caching , encryption , and compression . 
 a basic http request consists of a verb ( method ) and a resource ( endpoint ) . below are common http verbs : 
 verb description idempotent * safe cacheable 
 get reads a resource yes yes yes 
 post creates a resource or trigger a process that handles data no no yes if response contains freshness info 
 put creates or replace a resource yes no no 
 patch partially updates a resource no no yes if response contains freshness info 
 delete deletes a resource yes no no 
 * can be called many times without different outcomes . 
 http is an application layer protocol relying on lower - level protocols such as tcp and udp . 
 source ( s ) and further reading : http 
 what is http ? 
 difference between http and tcp 
 difference between put and patch 
 transmission control protocol ( tcp ) 
 source : how to make a multiplayer game 
 tcp is a connection - oriented protocol over an ip network . connection is established and terminated using a handshake . all packets sent are guaranteed to reach the destination in the original order and without corruption through : 
 sequence numbers and checksum fields for each packet 
 acknowledgement packets and automatic retransmission 
 if the sender does not receive a correct response , it will resend the packets . if there are multiple timeouts , the connection is dropped . tcp also implements flow

---
### 4. [Grokking the System Design Interview | #1 System Design Course](https://www.designgurus.io/course/grokking-the-system-design-interview)
*designgurus.io*  
**🌐 URL:** [https://www.designgurus.io/course/grokking-the-system-design-interview](https://www.designgurus.io/course/grokking-the-system-design-interview)

**Snippet**
> grokking the system design interview 
 ace your system design interview with the original grokking the system design interview course by design gurus – ex - faang hiring managers . with 30 k + 5 ★ rev...

**Summary**
> grokking the system design interview 
 ace your system design interview with the original grokking the system design interview course by design gurus – ex - faang hiring managers . with 30 k + 5 ★ reviews , this course will help you master scalability , distributed systems , and more . 
 level : beginner 
 study time : 
 20 h 
 lessons : 65 
 videos 
 4.7 
 best seller 
 course overview 
 grokking the system design interview is the original and most comprehensive course for system design interviews . created by ex - faang hiring managers from google , facebook , amazon , and microsoft , it distills years of real - world experience into a proven step - by - step framework that has helped over 100,000 software engineer to gain a deep mastery of system design interviews . whether you ’re a junior developer seeking to learn system design fundamentals or a seasoned engineer brushing up on distributed systems , this course ensures you’ll excel in any high - stakes interview . from caching and load balancing to database sharding and queue - based architectures , every concept is explained with clarity and practical examples . structured around a clear , repeatable methodology , the course shows you exactly how to tackle any system design question — from clarifying requirements and defining key components to handling performance trade - offs and scaling globally . the curriculum is continually updated to reflect the latest industry trends and best practices , ensuring you stay ahead of the curve . with real - world case studies , engaging illustrations , and a 4 . 7 - star average rating , “ grokking the system design interview ” remains the top resource for those who want to confidently learn system design . whether you ’re interviewing at a faang company or any tech - driven organization , this course gives you the inside . . . 
 what you'll learn 
 a proven approach to systematically tackle any system design question — from requirements gathering to final detailed solution . 
 master the core concepts of scalability , reliability , and performance to build robust , web - scale systems . 
 dive into caching strategies , load balancing methods , and database sharding to handle complex , large - scale scenarios . 
 get insider tips from ex - google , facebook , amazon , and microsoft interviewers on what top tech companies expect in system design interviews . 
 master the art of requirements engineering — from clarifying requirements to understanding constraints — for large - scale systems . 
 learn system design with real - world patterns : microservices , event - driven architectures , data partitioning , and more . 
 explore hands - on examples like designing instagram , uber - like services , and messaging platforms to gain practical experience . 
 understand how to choose the right technology stack , communication protocol ( rest vs. grpc ) , and hosting strategy ( cloud vs. on - prem ) . 
 course content 
 • 
 chapters • 
 65 
 lessons 
 1. 
 introduction to system design interview 
 4 lessons 
 what is a system design interview ? 
 understand the purpose of system design interviews , focusing on scalability , architecture decisions , trade - offs , and real - world problem - solving skills . 
 functional vs. non - functional requirements 
 master functional vs. non - functional requirements in system design , emphasizing scalability , performance , and trade - offs for building robust , real - world systems . 
 what are back - of - the - envelope estimations ? 
 master back - of - the - envelope estimation to assess scalability , performance , and trade - offs using quick calculations in system design interviews . 
 things to avoid during system design interview 
 learn what to avoid in system design interviews — like skipping trade - offs , ignoring requirements , poor communication , and rigid thinking . 
 2. 
 glossary of system design basics 
 19 lessons 
 system design basics 
 get familiar with core system design concepts , key architectural components , and building blocks of scalable , distributed software systems . 
 key characteristics of distributed systems 
 explore key characteristics of distributed systems , including scalability , reliability , availability , efficiency , fault tolerance , and manageability . 
 load balancing 
 learn how load balancing improves scalability and availability by distributing traffic using smart algorithms , health checks , and redundancy techniques . 
 caching 
 learn caching strategies to improve system performance , including cache types , eviction policies , read / write methods , and cache invalidation techniques . 
 data partitioning 
 learn data partitioning techniques — horizontal , vertical , hybrid — and partitioning criteria to scale databases , improve performance , and balance workloads . 
 indexes 
 understand how database indexes optimize query performance , enable efficient lookups , and impact write operations in large - scale systems . 
 proxies 
 learn how forward and reverse proxies improve security , caching , and traffic control by mediating communication between clients and servers . 
 redundancy and replication 
 learn how redundancy and replication improve system reliability and availability through failover strategies and synchronous , asynchronous , and semi - synchronous replication . 
 sql vs. nosql 
 compare relational and non - relational databases to understand schema flexibility , scalability , acid compliance , and ideal use cases for each . 
 cap theorem 
 understand why distributed systems must trade off between consistency , availability , and partition tolerance — only two of the three can be guaranteed at any time . 
 pacelc theorem 
 learn how pacelc extends cap by showing that even without partitions , distributed systems must balance latency and consistency in replicated environments . 
 consistent hashing 
 learn the architecture of scalable systems using consistent hashing for efficient data partitioning , replication , and dynamic node management . 
 long - polling vs websockets vs server - sent events 
 understand the system design of real - time communication protocols for scalable push - based updates between client and server . 
 bloom filters 
 explore space - efficient system design using bloom filters for fast , probabilistic membership checks with minimal memory and no false negatives . 
 quorum 
 design highly available distributed systems using quorum - based consistency models to coordinate read / write operations across replicated nodes . 
 leader and follower 
 understand the architecture of leader - based replication in distributed systems for consistent writes , fault tolerance , and coordinated data synchronization . 
 heartbeat 
 learn how distributed systems use heartbeat signals to detect server failures and maintain reliable request routing and availability . 
 checksum 
 explore how distributed systems ensure data integrity by using checksums to detect and prevent data corruption during transmission . 
 quiz 
 test your understanding of system design basic concepts . 
 3. 
 system design trade - offs new 
 22 lessons 
 importance of discussing trade - offs 
 demonstrating trade - offs shows maturity , critical thinking , and real - world design skills crucial for scalable , practical system architecture . 
 strong vs eventual consistency 
 explore consistency models in distributed systems , comparing strong and eventual consistency in terms of latency , availability , accuracy , and scalability trade - offs . 
 latency vs throughput 
 understand the difference between latency and throughput , and explore strategies to optimize response time and data processing capacity in systems . 
 acid vs base properties in databases 
 understand key differences between acid and base database models , balancing consistency , availability , scalability , and fault tolerance in distributed systems . 
 read - through vs write - through cache 
 explore read - through vs write - through caching strategies for improving read performance , data consistency , and integrity in scalable systems . 
 batch processing vs stream processing 
 compare batch vs stream processing in system design — complexity , resource efficiency , continuous vs scheduled data flow , and real - time responsiveness . 
 load balancer vs. api gateway 
 understand differences between load balancer and api gateway in scalable architectures — traffic distribution , request routing , api management , and availability . 
 api gateway vs direct service exposure 
 explore api gateway vs direct service exposure in distributed systems — compare centralized routing and security with direct access and reduced latency . 
 proxy vs. reverse proxy 
 understand how proxies and reverse proxies manage client and server traffic in distributed system design to improve security , caching , and scalability . 
 api gateway vs. reverse proxy 
 learn the difference between api gateways and reverse proxies in distributed systems , focusing on routing , security , load balancing , and orchestration . 
 sql vs. nosql 
 compare sql vs nosql databases based on schema structure , scalability , acid compliance , flexibility , and best use cases in system design . 
 primary - replica vs peer-to-peer replication 
 understand primary - replica vs peer-to-peer replication models , including data flow , consistency , scalability , fault tolerance , and use cases . 
 data compression vs data deduplication 
 learn the differences between data compression and deduplication for optimizing storage — focus , scope , efficiency , and best - use scenarios . 
 server - side caching vs client - side caching 
 understand server - side vs client - side caching — how they differ in location , control , performance impact , and best use across web apps . 
 rest vs rpc 
 learn the architectural differences between rest and rpc , including stateless resource handling vs. procedure calls , flexibility , scalability , and performance trade - offs . 
 polling vs. long - polling vs. websockets vs. webhooks 
 compare real - time delivery strategies — polling , long - polling , websockets , and webhooks — for scalable , event - driven system communication . 
 cdn usage vs direct server serving 
 learn when to use a content delivery network vs direct server hosting based on traffic scale , geographic distribution , and caching efficiency . 
 serverless architecture vs traditional server - based 
 learn serverless system design benefits like dynamic scaling and reduced ops versus traditional server hosting with full control and overhead . 
 stateful vs stateless architecture 
 explore the architecture of stateful vs stateless systems , comparing session handling , scalability , and design trade - offs for apis and web services . 
 hybrid cloud storage vs all - cloud storage 
 explore hybrid cloud vs all - cloud storage architectures , balancing scalability , compliance , and control for secure , flexible enterprise data strategies . 
 token bucket vs leaky bucket 
 learn how token bucket and leaky bucket algorithms manage network traffic shaping , rate limiting , and handling of bursty data . 
 read heavy vs write heavy system 
 explore how system design varies for read - heavy and write - heavy workloads with techniques like batching , replication , and data partitioning . 
 4. 
 system design problems 
 18 lessons 
 system design interviews - a step by step guide 
 system design master template 
 designing a url shortening service like tinyurl 
 designing pastebin 
 designing instagram 
 designing dropbox 
 designing facebook messenger 
 designing twitter 
 designing youtube or netflix 
 designing typeahead suggestion 
 designing an api rate limiter 
 designing twitter search 
 designing a web crawler 
 designing facebook ’s news feed 
 designing yelp or nearby friends 
 designing uber backend 
 designing ticketmaster 
 additional resources 
 5. 
 appendix 
 2 lessons 
 faqs 
 what is a system design interview ? 
 system design interview tests your ability to design scalable , efficient , and fault - tolerant systems . it involves creating a high - level architecture for real - world applications , such as social media platforms , e-commerce sites , or messaging systems . the focus is on your problem - solving , trade-off analysis , and design choices . 
 what is the best way to prepare for a system design interview ? 
 the best way to prepare is to learn key concepts like scalability , databases , caching , and apis . practice designing systems , such as url shorteners or chat apps , and explain your design choices . use resources like " grokking the system design interview " by design gurus . io for a structured approach . 
 is there coding in a system design interview ? 
 no , system design interviews typically don’t involve coding . these interviews focus on high - level architecture , system components , and design choices rather than writing code . however , you may need to discuss algorithms or pseudocode to explain certain design decisions . 
 how long is a system design interview ? 
 a system design interview typically lasts 45 - 60 minutes . during this time , you'll analyze requirements , propose a design , discuss trade - offs , and answer follow - up questions . efficient time management is crucial to cover all aspects of the design . 
 how to nail system design interviews ? 
 to excel in system design interviews , thoroughly understand the problem requirements , break down the system into its components , and focus on scalability , reliability , and trade - offs . practice designing real - world systems and use structured resources like " grokking the system design interview " to build your skills . 
 how to answer a system design question in an interview ? 
 to answer a system design question , start by clarifying the requirements , then outline the system 's high - level architecture . break the design into key components ( e . g . , database , apis , caching ) and explain trade - offs . address scalability , fault tolerance , and performance to show a complete understanding . 
 how to get prepared for a system design interview ? 
 prepare for a system design interview by learning fundamentals like databases , caching , load balancing , and distributed systems . practice designing systems , such as chat apps or e-commerce sites . use resources like " grokking the system design interview " for examples and guidance . 
 are system design interviews hard ? 
 yes , system design interviews can be challenging because they test complex topics like scalability , distributed systems , and trade - offs . however , with practice and understanding of common system design patterns , they become manageable . structured resources like " grokking the system design interview " can simplify preparation . 
 how to solve system design problems ? 
 to solve system design problems , start by understanding the requirements and constraints . break the system into components like storage , apis , and communication layers . focus on scalability , fault tolerance , and efficiency while discussing trade - offs and justifying your design choices . 
 how to practice system design concepts ? 
 practice system design concepts by studying real - world applications , such as url shorteners , social media platforms , and ride - sharing apps . break down their architectures , identify components , and explore trade - offs . utilize resources like " grokking the system design interview " and participate in mock interviews to gain hands - on experience . 
 what is expected in a system design interview ? 
 in a system design interview , you 're expected to analyze requirements , design a scalable and reliable architecture , and explain your choices . key topics include databases , apis , caching , load balancing , and trade - offs . clear communication and a structured approach are crucial . 
 how to answer a system design interview ? 
 to answer a system design interview , clarify requirements first . propose a high - level architecture , divide the system into its components , and explain how these components interact . discuss scalability , reliability , and trade - offs to demonstrate a thorough understanding of the design . 
 how to get good at system design ? 
 to excel in system design , focus on understanding core concepts such as scalability , databases , caching , and distributed systems . practice solving real - world design problems and learn from examples in resources like " grokking the system design interview . " consistent practice and feedback are key . 
 is a system design interview the same as a coding interview ? 
 no , a system design interview is different from a coding interview . coding interviews focus on solving algorithmic problems with code , while system design interviews evaluate your ability to create scalable and efficient architectures for real - world applications . both test different skill sets . 
 what questions are asked in a system design interview ? 
 design interviews often include questions like designing a url shortener , a messaging app , a ride - sharing platform , or an e-commerce system . you'll be asked to address scalability , database design , api structure , caching , and trade - offs in your solutions .

---
### 5. [How to Answer System Design Questions on Interviews?](https://javarevisited.substack.com/p/how-to-answer-system-design-questions)
*javarevisited.substack.com*  
**🌐 URL:** [https://javarevisited.substack.com/p/how-to-answer-system-design-questions](https://javarevisited.substack.com/p/how-to-answer-system-design-questions)

**Snippet**
> how to answer system design questions on interviews ? 
 3-step cheat sheet to answer any system design question on interview 
 hello guys , in last few articles i have shared a lot of resources like b...

**Summary**
> how to answer system design questions on interviews ? 
 3-step cheat sheet to answer any system design question on interview 
 hello guys , in last few articles i have shared a lot of resources like books , courses , websites , and interview problems to prepare for system design interview and today we are bringing in again a great topic , how to answer system design question on interview . 
 i have seen many people faltering and not performing to their potential due to not knowing how to articulate their answer to bring most important point quickly during system design interview . 
 and , today , we have aniket singh from 
 system design alamanac as a guest to share his experience on how you can overcome that hurdle . 
 over to aniket now : 
 designing systems is tough . a system design interview is tougher . 
 with little time to impress , it ’s vital to stay focused and avoid getting sidetracked by unimportant details . that 's why a structured framework is essential . 
 it keeps you on track and ensures you emphasize the right aspects of problem - solving , communication , and technical skills . 
 why does this matter ? 
 it can determine whether you ’re seen as a fit for junior or senior roles . 
 it also has a direct impact on your compensation . 
 this article will explore a proven framework . it will help you ask the right questions and focus on what matters in a system design interview . 
 by the way , if you are preparing for system design interviews and want to learn system design in depth then you can also checkout sites like bytebytego , design guru , exponent , educative , code mia . io , interview reddy . io and udemy which have many great system design courses 
 how to answer system design interview question ? 
 before we dive deep , here 's a quick cheat sheet for approaching any system design question . 🧷

---
### 6. [Low Level Design Primer](https://awesomeopensource.com/project/prasadgujar/low-level-design-primer)
*awesomeopensource.com*  
**🌐 URL:** [https://awesomeopensource.com/project/prasadgujar/low-level-design-primer](https://awesomeopensource.com/project/prasadgujar/low-level-design-primer)

**Snippet**
> low level design primer 
 dedicated resources for the low - level system design . learn how to design and implement large - scale systems . prep for the system design interview . 
 categories   >   le...

**Summary**
> low level design primer 
 dedicated resources for the low - level system design . learn how to design and implement large - scale systems . prep for the system design interview . 
 categories   >   learning resources   >   interview 
 stars 
 5 , 543 
 license 
 no license specified 
 open issues 
 25 
 most recent commit 
 4 months ago 
 categories 
 learning resources   >   interview 
 software architecture   >   design pattern 
 learning resources   >   interview questions 
 learning resources   >   interview preparation 
 software architecture   >   system design 
 repo

---
### 7. [Salman Khan - Wikipedia](https://en.wikipedia.org/wiki/Salman_Khan)
*en.wikipedia.org*  
**🌐 URL:** [https://en.wikipedia.org/wiki/Salman_Khan](https://en.wikipedia.org/wiki/Salman_Khan)

**Snippet**
> salman khan 
 salman khan [ a ] ( hindi : [ səlˈmɑːn ˈxɑːn ] ; born 27 december 1965 ) is an indian actor , film producer , and television personality who predominantly works in hindi films . in a car...

**Summary**
> salman khan 
 salman khan [ a ] ( hindi : [ səlˈmɑːn ˈxɑːn ] ; born 27 december 1965 ) is an indian actor , film producer , and television personality who predominantly works in hindi films . in a career spanning over three decades , his awards include two national film awards as a film producer , and two filmfare awards as an actor . [ 3 ] he has been cited in the media as one of the most popular and commercially successful actors of indian cinema . [ 4 ] [ 5 ] forbes included him in listings of the highest - paid celebrities in the world , in 2015 and 2018 . [ 6 ] [ 7 ] [ 8 ] he has starred in the annual highest - grossing hindi films of 10 individual years , the highest for any actor . [ 9 ] 
 salman khan 
 khan in 2023 
 born 
 ( 1965 - 12 - 27 ) 
 27 december 1965 ( age 59 ) 
 indore , madhya pradesh , india 
 occupations 
 actor 
 film producer 
 television presenter 
 years active 
 1988 –present 
 works 
 parents 
 salim khan ( father ) 
 helen ( step - mother ) 
 relatives 
 sohail khan ( brother ) 
 arbaaz khan ( brother ) 
 alvira khan agnihotri ( sister ) 
 family 
 salim khan family 
 awards 
 signature 
 khan began his acting career with a supporting role in biwi ho to aisi ( 1988 ) , followed by his breakthrough with a leading role in sooraj barjatya 's romantic drama maine pyar kiya ( 1989 ) , for which he was awarded the filmfare award for best male debut . he established himself with several commercially successful films , including lawrence d'souza 's romantic drama saajan ( 1991 ) , barjatya 's family dramas hum aapke hain koun . . ! ( 1994 ) and hum saath - saath hain ( 1999 ) , the action film karan arjun ( 1995 ) and the comedy biwi no . 1 ( 1999 ) . after a period of mixed success in romantic comedy , musicals and tragedy drama in 2000 s , khan resurrected his screen image with the action film wanted ( 2009 ) , and achieved greater stardom the following decade by starring in the top - grossing action films dabangg ( 2010 ) , ready ( 2011 ) , bodyguard ( 2011 ) , ek tha tiger ( 2012 ) , dabangg 2 ( 2012 ) , kick ( 2014 ) , and tiger zinda hai ( 2017 ) , and the dramas bajrangi bhaijaan ( 2015 ) and sultan ( 2016 ) . this was followed by a series of poorly - received films , except tiger 3 ( 2023 ) . 
 in addition to his acting career , khan is a television presenter and promotes humanitarian causes through his charity , being human foundation . [ 10 ] he has been hosting the reality show bigg boss since 2010 . [ 11 ] khan 's off - screen life is marred by controversy and legal troubles . in 2015 , he was convicted of culpable homicide for a negligent driving case in which he ran over five people with his car , killing one , but his conviction was set aside on appeal . [ 12 ] [ 13 ] [ 14 ] [ 15 ] on 5 april 2018 , khan was convicted in a blackbuck poaching case and sentenced to five years imprisonment . [ 16 ] [ 17 ] on 7 april 2018 , he was out on bail while an appeal was ongoing . [ 18 ] 
 contents 
 1 early life and ancestry 
 2 career 
 2.1 1988 – 1993 : debut and breakthrough 
 2.2 1994 – 1999 : rise to prominence 
 2.3 2000 – 2009 : sporadic success 
 2.4 2010 – 2017 : superstardom 
 2.5 2018 –present : decline 
 3 other work 
 3.1 production 
 3.2 salman khan films 
 3.3 television 
 3.4 brand endorsements 
 3.5 business 
 4 philanthropy and service 
 5 personal life 
 6 in the media 
 7 controversies 
 7.1 hit - and - run case 
 7.2 relationship with aishwarya rai 
 7.3 blackbuck hunting and arms act violations cases 
 7.4 remarks about 26/11 attacks 
 7.5 tweets on yakub memon 
 7.6 interference in jiah khan suicide case 
 7.7 threats from lawrence bishnoi gang 
 8 discography 
 9 accolades 
 11 notes 
 12 references 
 early life and ancestry 
 salman khan ( between ) with brothers arbaaz khan ( left ) and sohail khan ( right ) 
 salman khan is the eldest son of screenwriter salim khan and his first wife sushila charak , who adopted the name salma . [ 19 ] [ 20 ] born to a muslim father and hindu mother on 27 december 1965 , salman khan was brought up in both faiths . [ 21 ] in 1981 , when salim married the actress helen , the children 's relationship with their father grew hostile and only recovered years later . [ 22 ] 
 salman khan 's paternal great - grandparents are believed to be alakozai pashtuns from afghanistan who immigrated to indore state , indore residency ( now in madhya pradesh ) , british india in the mid - 1800 s ; [ 23 ] [ 24 ] [ 25 ] [ 26 ] however , jasim khan in his biography of the actor states that his ancestors belonged to the akuzai sub - tribe of the yusufzai pashtuns from malakand in the swat valley of north - west frontier province , british india ( present - day khyber pakhtunkhwa , pakistan ) . [ 27 ] his grandfather abdul rashid khan was a deputy inspector general of the indore state who was awarded the diler jung award of the holkar times . [ 28 ] khan 's mother is a homemaker , [ 29 ] [ 30 ] whose father baldev singh charak , [ 31 ] a dogra rajput , [ 32 ] comes from jammu in jammu and kashmir [ 31 ] [ 33 ] and whose maratha mother comes from maharashtra . [ 30 ] khan can also speak marathi , in addition to hindi and english . [ 34 ] he has two brothers , arbaaz khan and sohail khan ; and two sisters , alvira khan agnihotri , who is married to actor - director atul agnihotri , and an adopted sister arpita , who is married to actor aayush sharma . [ 35 ] 
 salman finished his schooling at st. stanislaus high school in bandra , mumbai , as did his younger brothers arbaaz and sohail . previously , he studied at the scindia school in gwalior for a few years along with his younger brother arbaaz . [ 36 ] he attended st. xavier 's college in mumbai but dropped out . [ 37 ] 
 career 
 1988 – 1993 : debut and breakthrough 
 salman khan made his on - screen debut in 1988 with a supporting role in the successful film biwi ho to aisi , which had rekha in the lead role . [ 38 ] [ 39 ] the following year , he played the main lead in sooraj barjatya 's romantic musical maine pyar kiya opposite bhagyashree . [ 40 ] the film backed up with chartbuster music went on to become an all time blockbuster at the box office and made khan a star . [ 41 ] it also won him filmfare award for best male debut . [ 42 ] 
 he began the new decade with deepak shivdasani 's action drama film baaghi : a rebel for love ( 1990 ) , which he also wrote . [ 43 ] baaghi opened to positive critical reception and was a box office hit . [ 44 ] khan 's dream run continued in 1991 with two of his films , sanam bewafa and saajan emerging blockbusters and their soundtracks proving to be two of the best - selling hindi film albums of that year . [ 45 ] [ 46 ] his other releases of the year , kurbaan and patthar ke phool also performed well commercially and proved to be major commercial successes . [ 45 ] khan followed it with a downturn as all of his films for the next two years , including biggies jaagruti ( 1992 ) and chandra mukhi ( 1993 ) were critical and commercial failures . 
 1994 – 1999 : rise to prominence 
 in 1994 , khan appeared in rajkumar santoshi 's andaz apna apna , co - starring aamir khan . at the time of its release , the film failed at the box office but has gained a cult status over the years . [ 47 ] later in the year he collaborated with director sooraj barjatya in the romance hum aapke hain koun . . ! co - starring madhuri dixit . during the 1995 awards season , the film won 3 filmfare awards for best film , best director and best actress . it also won the national award for being the most popular film of the year . [ 48 ] earning more than ₹ 2 billion [ 49 ] [ 50 ] [ 51 ] 
 ( $ 63 . 8 million ) worldwide , the film became the biggest bollywood hit of the year and the highest - grossing indian film up until then . [ 52 ] it is one of the films on box office india 's list of " biggest blockbusters ever in hindi cinema " . [ 53 ] in 2006 , it was still the fourth - highest grossing bollywood film ever , according to box office india . [ 54 ] in 1995 he starred in rakesh roshan 's karan arjun alongside shah rukh khan . the two played brothers who are reincarnated after being killed by family enemies . his role as karan earned him a nomination for the 1995 filmfare best actor award . [ 55 ] 
 in 1996 , khan performed in sanjay leela bhansali 's directional debut khamoshi : the musical . [ 56 ] he also appeared in and raj kanwar 's action hit jeet . [ 57 ] he had two releases in 1997 : judwaa and auzaar . the former was a comedy directed by david dhawan where he played a dual role of twins separated at birth . [ 58 ] 
 khan worked in five different films in 1998 , his first release being the romantic comedy film pyaar kiya to darna kya opposite kajol , one of the biggest commercial successes of that year . [ 59 ] this was followed by the moderately successful drama jab pyaar kisi se hota hai [ 60 ] where khan played a young man who has to take a child who claims to be his son , under his custody . khan 's performance in the film earned favourable critical review . [ 61 ] he also had an extended cameo in karan johar 's directorial debut , kuch kuch hota hai , which earned him critical acclaim and a second filmfare award under the best supporting actor category . [ 62 ] 
 in 1999 , khan starred in three films : hum saath - saath hain , biwi no . 1 , [ 63 ] and hum dil de chuke sanam alongside aishwarya rai and ajay devgan , which earned him another best actor nomination at the filmfare awards . [ 64 ] his performance in the film was reviewed favourably , with sharmila telikum of rediff noting , " salman is endearing . he does tend to ham in the dramatic scenes , but looks very comfortable doing comedy and those romantic scenes . " [ 65 ] 
 2000 – 2009 : sporadic success 
 despite some big successes , most of khan 's films were commercial failures during this period , including chal mere bhai ( 2000 ) , kahin pyaar na ho jaaye ( 2000 ) , tumko na bhool pa ayenge ( 2002 ) , yehi hai jalwa ( 2002 ) , garv : pride & honour ( 2004 ) , phir milenge ( 2004 ) , dil ne jise apna kahaa ( 2004 ) , lucky - no time for love ( 2005 ) , kyon ki ( 2005 ) , shaadi karke phas gaya yaar ( 2006 ) , jaan - e - mann ( 2006 ) , baabul ( 2006 ) , salaam - e - ishq ( 2007 ) , god tussi great ho ( 2008 ) , yuvvraaj ( 2008 ) , main aurr mrs. khanna ( 2009 ) , and london dreams ( 2009 ) . [ 66 ] 
 in 2001 , khan appeared in chori chori chupke chupke , which addressed the issue of surrogate childbirth . in it , khan played the role of a rich industrialist who hires a surrogate mother after his wife becomes infertile . [ 67 ] sukanya verma of reddit wrote that the film had an absurd storyline , but also spontaneous performances from the cast , helping to minimise its other flaws . [ 67 ] in 2002 , he starred in hum tumhare hain sanam . [ 68 ] 
 for tere naam ( 2003 ) , taran adarsh said of him , " salman khan is exceptional in a role that fits him to the t. he breathes fire in sequences that demand uneasiness . but beneath the tough exterior lies a vulnerable person and this facet in particular comes to the fore in the latter reels . his emotional outbursts are splendid . . . " [ 69 ] director satish kaushik considered khan 's work in the film to be strong , saying of it , " salman has given a great performance in the film . he 's given an intense , unconventional performance . i believe this is the best performance of salman' 's career so far . " [ 70 ] his career progressed with comedies like mujhse shaadi karogi ( 2004 ) and no entry ( 2005 ) . [ 60 ] [ 71 ] 
 khan ( right ) with saif ali khan at an event for world aids day in 2007 
 khan started 2007 with the ensemble film salaam - e - ishq . he appeared in the hollywood movie , marigold opposite american actress ali larter . the film told the love story of an indian man and an american woman . [ 72 ] 
 khan hosted the second season of the game show 10 ka dum during 2009 . according to a 2008 report from biz asia uk , the show earned enough target rating points ( trps ) for sony entertainment television to regain its third position in the indian television ratings . [ 73 ] 
 he appeared in wanted ( a remake of 2006 telugu super hit film pokiri ) , directed by choreographer turned director prabhu deva . the film received mixed reviews . taran adarsh from bollywood hungama rated it 4 of 5 stars noting , " wanted rides on salman khan 's star power . he may not be the best actor in town , but in a film like wanted , in a role that seems like an extension of his personality , you can't think of anyone else enacting this role with flourish . " [ 74 ] raja sen from rediff gave a rating of 2/5 and said , " the writing is both amateurish and crass , while the songs are plain hideous . . . khan might be having fun , but the fact a film like wanted underscores is how badly bollywood needs a breed of younger leading men . and how the existing lot need roles that fit . " [ 75 ] he appeared in two other films that year , main aurr mrs khanna and london dreams . [ 76 ] [ 77 ] 
 2010 – 2017 : superstardom 
 khan 's first release of 2010 was anil sharma 's film veer . [ 78 ] in his second release of 2010 , dabangg , khan played the role of a fearless cop with comic effect in the film . [ 79 ] the film was noted by the economic times as being remarkable for its commercial success , despite having an " accent on inanity . . . " and " . . . complete incoherence in terms of plot and credibility . " [ 80 ] the times also noted industry experts attributed the popularity of the film to khan 's presence , stating that they " ascribe its allure to the star charisma of salman khan , who has managed to carry off the over - the - top articulation of chulbul pandey with unbridled enthusiasm and zeal . " [ 80 ] 
 dabangg won several awards , including the national film award for best popular film providing wholesome entertainment . [ 81 ] it was later remade in tamil and telugu . [ 82 ] the film was produced by his brother arbaaz khan . [ 81 ] it was reported by the hindustan times that the film opened to full houses across the country . [ 83 ] [ 84 ] khan received a star screen award for best actor [ 85 ] and a stardust award for star of the year – male for his performance . [ 86 ] he was also nominated for his sixth filmfare award for best actor . [ 87 ] anupama chopra from ndtv wrote about his performance : "it 's the role of a lifetime and salman khan bites into it like a starving man devours a feast . he inhabits it fully , strutting and swaggering and even , spoofing himself . " [ 88 ] 
 khan 's first release of 2011 was ready ( a remake of the 2008 telugu film of the same name ) . ready held a record for being the second highest grossing bollywood film of 2011 . [ 89 ] [ 90 ] he next appeared in bodyguard , a remake of the 2010 malayalam film of the same name . the film was not well received by critics , though it became india 's highest - earning film of the year . [ 91 ] 
 khan with katrina kaif at the launch of ek tha tiger 's first song ' mashallah ' 
 khan 's first release of 2012 was ek tha tiger where he starred opposite katrina kaif and acted as an indian spy . the film garnered positive reactions from critics [ 92 ] while opening to extremely strong box office collections . [ 93 ] the film marks his first association with yash raj films . [ 94 ] 
 khan starred in dabangg 2 , the sequel of dabangg , in 2012 under the production of arbaaz khan . dabangg 2 eventually emerged as a huge financial success with revenues of ₹ 2 . 5 billion ( us $ 46 . 78 million ) globally . [ 95 ] 
 after a one - year gap , khan 's first release of 2014 was jai ho ( an official remake of the 2006 telugu film stalin ) , which he starred in opposite daisy shah . movie became another hit film for salman khan . his second release , kick , a remake of a telugu film has entered the rs . 2 billion club in india on eid . he also sang the " hangover " song from the movie . [ 96 ] 
 khan 's first film of 2015 , bajrangi bhaijaan , which was released on eid , received acclaim from critics and the public and broke several box office records upon release . the film , grossing ₹ 1 . 84 billion ( us $ 28 . 68 million ) on its first week beat the previous record of pk . [ 97 ] this was khan 's first and the second bollywood film after pk to enter the 300 club . [ 98 ] the film became the second highest - grossing bollywood film in india and worldwide at that time , with a collection of over 6 billion . [ 99 ] bajrangi bhaijaan crossed 300 crore within 20 days of its release and became the second highest grosser to date in india , [ 100 ] [ 101 ] while his second film , prem ratan dhan payo , a diwali release received mixed reviews from critics and the public and broke several box office records upon release . the film became the ninth consecutive film of salman khan to gross over 1 billion . the film grossed ₹ 1 . 73 billion ( us $ 27 million ) on its first week . by 25 november , the film collected ₹ 2 . 01 billion . with this , khan become the only actor to give three back - to - back movies which collected more than ₹ 2 billion ( us $ 31 . 18 million ) net domestically . [ 102 ] he became the only actor to collect over ₹ 5 billion ( us $ 77 . 94 million ) domestic net in a single year in india . [ 103 ] [ 104 ] [ 105 ] [ 106 ] 
 khan 's first film of 2016 , yet again another eid release , sultan , directed by ali abbas zafar for yash raj films received positive reviews from both critics and the public breaking several box office records . the film opened on an average of 70 % audience occupancy [ 107 ] and grossed approximately ₹ 365 million ( us $ 5 . 43 million ) on its opening day . [ 108 ] the film also collected another ₹ 74 . 86 ( us $ 1 . 11 ) on its first weekend , bringing its total first week collections to an approximate of ₹ 2 . 08 billion ( us $ 30 . 95 million ) . [ 109 ] by the end of its second week , the film had grossed an estimated ₹ 2 . 78 billion ( us $ 41 . 37 million ) [ 110 ] and later became the second film of khan to net over 300 crores . [ 111 ] as of 9 august , the film grossed ₹ 5 . 83 billion ( us $ 86 . 76 million ) worldwide . [ 112 ] 
 in june 2017 , khan appeared in tubelight , his third collaboration with kabir khan after ek tha tiger and bajrangi bhaijaan . this film also starred khan 's real - life brother sohail khan . despite its high expectations , the film failed to do well at the box office . [ 113 ] khan 's second release of 2017 tiger zinda hai , a sequel to ek tha tiger [ 114 ] grossed ₹ 190 crore worldwide in its opening weekend . [ 115 ] as of 23 january 2018 , the film has grossed ₹ 5 . 52 billion ( us $ 84 . 76 million ) worldwide , [ 116 ] including ₹ 4 . 28 billion ( us $ 65 . 72 million ) in india [ 117 ] and ₹ 1 . 23 billion ( us $ 18 . 89 million ) overseas . [ 118 ] 
 2018 –present : decline 
 in 2018 , khan played the leading role in his only release of the year , race 3 , an action film . the film had an ensemble cast including anil kapoor , bobby deol , jacqueline fernandez and others . the film was critically panned for its storyline , weak performances and climax . box office india declared it an " average " grosser . [ 119 ] khan starred in bharat that was released on 5 june 2019 and dabangg 3 that was released on 20 december 2019 . [ 120 ] [ 121 ] while bharat had an average run , dabangg 3 flopped . [ 122 ] [ 123 ] meanwhile , kick 2 which was scheduled to release around christmas 2019 had been delayed . [ 124 ] [ 125 ] he starred in radhe which released on 13 may 2021 to negative reviews . his next film , antim : the final truth , where he played the second lead , received mixed reviews from critics , but the performance of khan was well appreciated . the film was a box office disappointment . [ 126 ] 
 he next did a cameo in pathaan , reprising his eponymous role as tiger . the same year , he starred in kisi ka bhai kisi ki jaan , the remake of veeram , directed by farhad samji . [ 127 ] the film received mixed reviews and turned out to be another failure for the actor . [ 128 ] khan next reprised his role as tiger in tiger 3 ( 2023 ) , as part of the yrf spy universe . [ 129 ] made on a budget of ₹ 300 crore ( us $ 35 million ) , [ 130 ] the movie went on to gross over ₹ 466 . 63 crore ( us $ 55 million ) worldwide to emerge a hit venture . [ 131 ] despite the success , tiger 3 underperformed compared to its prequels . [ 132 ] after making cameos in singham again and baby john in 2024 , khan starred in ar murugadoss ' action film sikandar ( 2025 ) . chirag segal of news18 praised khan 's action sequences and chemistry with rashmika mandanna , who is 31 years his junior , but noted his limited emotional range in dramatic scenes . [ 133 ] rahul desai of the hollywood reporter was critical of the movie and khan 's performance as " alpha - male - saviour " , stating that "khan 's version of reality is too detached from storytelling to affect the viewers . " [ 134 ] it became khan 's biggest flop since yuvvraaj ( 2008 ) . [ 135 ] 
 other work 
 production 
 in 2011 , he launched his own production company called skbh productions ( salman khan being human productions ) . money generated from film productions will be donated to the being human organisation . the first film made under the banner was the kids ' entertainer chillar party , which went on to win 3 national awards for best children 's film , best original screenplay and best child artist ( s ) . [ 136 ] [ 137 ] khan has been the brand ambassador of bmc 's open defecation free drive since 2016 . [ 138 ] 
 salman khan films 
 in 2014 , he launched another production house called skf ( salman khan films ) . the first film released under this banner was the canadian film dr. cabbie . the film earned $ 350 , 452 on its opening day at the box office . the next films under this banner were hero , in which he also sang the title song " hero " [ 139 ] [ 140 ] by nikhil advani , which starred aditya pancholi 's son suraj pancholi and sunil shetty 's daughter athiya shetty ; and bajrangi bhaijaan by kabir khan starring khan himself with kareena kapoor and nawazuddin siddiqui . [ 141 ] 
 films produced under salman khan films 
 year title director notes 
 2011 chillar party nitesh tiwari , 
 vikas bahl produced with the banner salman khan being human productions . 
 it won the 2011 national film award for best children 's film . [ 142 ] along with this , it also won 2011 national film award for best child artist and best screenplay ( original ) . 
 2014 dr. cabbie jean-françois pouliot 
 2015 bajrangi bhaijaan kabir khan national film award for best popular film providing wholesome entertainment 
 hero nikhil advani remake of hero 
 2017 tube light kabir khan adaptation of little boy 
 2018 race 3 remo d'souza 
 loveyatri abhiraj minawala unofficial remake of devadasu 
 2019 notebook nitin kakkar adaptation of the teacher 's diary 
 bharat ali abbas zafar adaptation of ode to my father 
 dabangg 3 prabhu deva 
 2021 kaagaz satish kaushik based on lal bihari " mritak " 
 radhe prabhu deva adaptation of the outlaws 
 antim : the final truth mahesh manjrekar remake of mulshi pattern 
 2023 kisi ka bhai kisi ki jaan farhad samji remake of veeram 
 farrey soumendra padhi remake of bad genius 
 2025 sikandar ar murugadoss released on eid al-fitr 2025 
 television 
 khan hosting 10 ka dum . 
 in 2008 khan hosted 10 ka dum . the show was extremely popular and was at the number one spot in ratings in india . it garnered an average tvr of 2.81 and a peak rating of 4 . 5 , leaving behind shah rukh khan 's kya aap paanchvi paas se tej hai ? with an average rating of 1.37 tvr and a peak rating of 2.3 and hrithik roshan 's junoon – kuch kar di khane ka on ndtv imagine with an average tvr of 0.76 and a peak rating of 1 . 1 . [ 143 ] according to reports , the show helped sony tv regain its third position in the indian television ratings . [ 144 ] he again hosted the show in 2009 and thus won the best anchor award for 10 ka dum in 2008 and 2009 . in 2010 khan hosted bigg boss 4. the show was widely accepted and appreciated due to khan 's hosting and surpassed the famous hosting of amitabh bachchan . [ 145 ] 
 the grand finale episode on 8 january 2011 , received a trp of 6 . 7 , which was the highest among the finales of other indian reality shows such as kaun banega crorepati , rahul dulhaniya le jayega , masterchef and did –li ' l masters . [ 146 ] due to high trp khan again hosted bigg boss 5 along with sanjay dutt in 2011 and because of huge acceptance he also hosted bigg boss 6 and bigg boss 7 in 2012 and 2013 . in 2013 khan hosted the star guild award for the first time . [ 147 ] [ 148 ] khan also hosted bigg boss 8 , bigg boss 9 , bigg boss 10 , bigg boss 11 , bigg boss 12 , bigg boss 13 , bigg boss 14 , bigg boss 15 , bigg boss 16 bigg boss 17 , and bigg boss 18 . 
 brand endorsements 
 khan was associated as a brand when he had advertised for campa cola , [ 149 ] limca soft drink , [ 150 ] hero honda bikes [ 151 ] and double bull shirts [ 152 ] well before he entered the film industry . even after he became a superstar , he never showed interest in promoting himself as a brand , but he was signed for thums up in 2002 and continued until its contract was over . later akshay kumar replaced khan . later he was the brand ambassador of the soft drink mountain dew with whom he ended the contract in december 2010 , [ 153 ] and he has once again promoted thums up but ended contract soon after . [ 154 ] [ 155 ] he is now brand ambassador for pepsi . [ 156 ] he has also become the brand ambassador for the travel website , yatra , which also made him a shareholder . [ 157 ] he is also the face of the history channel and the new brand ambassador for suzuki motorcycles , [ 158 ] previously he endorsed red tape shoes , [ 159 ] and now he is endorsing relaxo hawaii . khan is also the brand ambassador of the detergent brand wheel . [ 160 ] he has also appeared for chlormint , a brand of gum , with his brother sohail khan . the actor has also done a jewellery brand , sangini with kareena kapoor . [ 161 ] apart from innerwear brand dixcy scott and tablet blackberry playbook , britannia ' s tiger biscuits is the latest endorsement of salman . [ 162 ] he is also the face of ranbaxy 's revital , replacing yuvraj singh . [ 163 ] apart from these brands , he also has rotomac pen and sf sonic batteries endorsements . [ 164 ] now khan has been roped in as the brand ambassador of dubai - based fashion label splash , [ 165 ] and has been also signed as the brand ambassador for astral limited the leading manufacturers of plumbing and drainage systems in the country . [ 166 ] khan is also the brand ambassador of civic body bombay municipal corporation 's ( bmc ) drive against open defecation in mumbai , [ 167 ] and has been also signed by the security and surveillance brand , cp plus as their cctv ambassador . [ 168 ] khan is now the second ambassador along with amitabh bachchan for ' emami healthy & tasty ' , ' himani best choice ' and ' rasoi ' , apart from ' bake magic ' . [ 169 ] 
 business 
 the renowned bollywood actor , made a significant investment in the property sector in 2012 when he purchased a commercial property on linking road for approximately 120 crore indian rupee . in 2017 , the property gained prominence as it became the location for future retail 's food hall , which was owned by kishore biyani 's future group . an agreement was struck for a monthly rental of 80 lakh indian rupees over a five - year period . [ 170 ] however , due to non - payment of dues , a legal dispute a rose , ultimately resulting in a ruling by the national company law appellate tribunal ( nclat ) in favour of salman khan . fast forward to 2023 , and the property has found new occupants in the form of food square , a luxury gourmet retail brand owned by landcraft retail private limited . the co - founders of food square , lalit jhawar and mayank gupta , successfully secured $ 3 . 6 million in funding from notable investors , including masaba gupta , mukul agrawal , purple style labs , sanket parekh ( from the pidilite family ) , rahul kayan ( smifs ) , and harminder sahni . [ 171 ] food square now rents out the salman khan - owned property for a monthly fee of 1 crore indian rupees , [ 170 ] marking a new chapter in the property 's history . 
 philanthropy and service 
 main article : being human foundation 
 khan at his being human show with actresses , ( l - r ) kareena kapoor , rani mukerji , preity zinta , katrina kaif , karisma kapoor and priyanka chopra 
 khan has been involved in several charities during his career . [ 172 ] he has started an ngo called being human which sells t-shirts and other products online and in stores . a portion of the revenues go to a worthy cause supporting the underprivileged . [ 173 ] being human foundation is a registered charitable trust set up by khan to help the underprivileged . in its early days , khan set up and funded the foundation using his own money . the foundation has two focus areas : education and healthcare . to increase the reach and corpus of the foundation , salman khan has undertaken initiatives such as being human art , being human merchandise and being human gitanjali gold coins . [ 174 ] 
 in 2011 , he launched his own production company called skbh productions ( salman khan being human productions ) . money which will be generated from film productions will be donated to being human . the first film made under the banner was the kids ' entertainer chillar party , which went on to win 3 national awards for best children 's film , best original screenplay and child artist 's award . [ 136 ] [ 137 ] 
 in january 2012 , khan offered to pay ₹ 4 million ( us $ 74 , 853 . 66 ) to release about 400 prisoners from about 63 prisons in the state of uttar pradesh via his ngo . the prisoners had completed their terms , but due to economic reasons , they were unable to pay legal fines for their charges . [ 175 ] 
 in july 2015 , khan offered to donate the profits of his most successful film bajrangi bhaijaan to poor farmers all over india . [ 176 ] the film 's producers khan and rockline venkatesh both have mutually decided to donate the profits of their film to them . the film 's director kabir khan and salman 's sister alvira agnihotri met maharashtra 's revenue minister eknath khadse to discuss the same . [ 177 ] 
 khan met with his 11 - year - old fan from pakistan , abdul basit who had severe jaundice at the time of his birth ; he was diagnosed to have crigler najjar syndrome and was admitted to the apollo hospital , delhi . the youngster wished to meet the " bajrangi bhaijaan " star khan . [ 178 ] [ 179 ] [ 180 ] [ 181 ] [ 182 ] [ 183 ] 
 khan was one of the nine personalities invited by india 's prime minister narendra modi to promote the message of cleanliness and swachh bharat abhiyan when it was launched on 2 october 2014 . [ 184 ] in 2016 , he agreed to campaign for cleanliness and eradicating open defecation for the brihan mumbai municipal corporation . [ 185 ] his efforts to clean the streets of karjat , a small town near mumbai were appreciated by the prime minister in a tweet . [ 186 ] 
 personal life 
 khan resides in galaxy apartments , bandra , mumbai . he also has a 150 - acre plot in panvel which has 3 bungalows , a swimming pool and a gym . [ 187 ] he is a fitness enthusiast and maintains a strict regimen . [ 187 ] [ 188 ] 
 khan has never married . in 1999 , he began dating bollywood actress aishwarya rai ; their relationship was often reported in the media until the couple separated in 2001 . khan started dating actress katrina kaif . after years of speculations , kaif admitted in an interview in 2011 that she had been in a serious relationship with khan for several years , but it ended in 2010 . [ 189 ] sangeeta bijlani and somy ali were also in serious relationships with khan . [ 190 ] 
 since 2012 , khan is in a relationship with romanian actress iulia vântur . [ 191 ] 
 in august 2011 , khan admitted that he suffers from trigeminal neuralgia , a facial nerve disorder commonly known as the " suicide disease " . the disorder occurs due to the inflammation of trigeminal nerve in the face . [ 192 ] in an interview , he said that he has been quietly suffering from it for the past seven years , but now the pain has become unbearable . he said that it has even affected his voice , making it much harsher . [ 193 ] 
 khan identifies as both muslim and hindu , commenting that , "i 'm hindu and muslim both . i 'm bharatiya ( an indian ) " . [ 21 ] he explained , " my father is muslim and my mother is hindu " . [ 194 ] 
 in the media 
 salman khan promoting his production venture chillar party 
 salman khan 's name is often shortened to sk in the media , also reflected in the name of his gym brand " sk - 27 gym " and his film company skf ( salman khan films ) . [ 195 ] [ 196 ] he is also known by his nickname " bhai " or " bhaijaan " ( meaning brother ) , referenced in the title of his films bajrangi bhaijaan ( 2015 ) and kisi ka bhai kisi ki jaan ( 2023 ) . [ 197 ] 
 in 2004 khan was ranked as the 7th best looking man in the world by people magazine of the usa . [ citation needed ] in 2008 his lifelike wax statue was installed in london 's madame tussauds museum ; likewise , in 2012 another wax statue of him was installed in madame tussauds new york museum . in 2010 people magazine , india declared him the sexiest man alive . [ citation needed ] in 2011 , 2012 and 2013 he was declared the times of india ' s most desirable man in 2 nd , 1st and 3rd place respectively . [ 198 ] in august 2013 he was declared india 's most searched celebrity online . [ 199 ] khan topped the forbes india charts for 2014 , in terms of both fame and revenues . [ 200 ] according to the forbes 2015 list ' celebrity 100 : the world 's top - paid entertainers 2015 ' , khan was the highest ranked indian in the 71st rank with earnings of $ 33 . 5 million . [ 7 ] 
 khan was ranked seventh in forbes ' s first global list of highest - paid actors in the world , with earnings of us $ 33 . 5 million , which placed him higher than such hollywood actors as johnny depp , brad pitt , leonardo dicaprio and dwayne the rock johnson . [ 201 ] [ 202 ] [ 203 ] [ 204 ] in 2015 , he was ranked ahead of us president barack obama in the list of the most admired personalities of india . [ 205 ] in september 2015 , the international business times declared him " the most attractive personality " in india . [ 206 ] he also became the only actor to collect over ₹ 5 billion ( us $ 77 . 94 million ) domestic net in a single year in india . [ 103 ] [ 104 ] [ 105 ] [ 106 ] khan was ranked the second " most googled indian of 2015 " and first among bollywood actors . [ 207 ] [ 208 ] on 24 april 2016 the indian olympic association appointed the actor as the indian olympic contingent 's goodwill ambassador for the 2016 summer olympics . [ 209 ] in august 2017 , he was identified as the ninth highest - paid actor in the world , and the first place highest - paid in india according to forbes india . [ 210 ] 
 controversies 
 hit - and - run case 
 on 28 september 2002 , khan was arrested for rash and negligent driving after his car ran into a bakery in mumbai ; one person who was sleeping on the pavement outside the bakery died and three others were injured in the accident . [ 211 ] charges of culpable homicide were laid against him , but later dropped . [ 212 ] on 24 july 2013 , he was formally charged with culpable homicide in the case , to which he pleaded not guilty . [ 213 ] 
 on 6 may 2015 , khan was found guilty of all charges in the case . the bombay sessions court concluded that khan was driving the car under the influence of alcohol , causing the death of one and serious injury to four homeless persons . an rto officer confirmed that khan did not have a driving licence until 2004 . [ 214 ] sessions judge dw deshpande convicted the actor for culpable homicide not amounting to murder and sentenced him to five years in prison . [ 215 ] later in the same day , khan , being represented by senior counsel amit desai was granted bail by the bombay high court till 8 may 2015 , [ 216 ] [ 217 ] [ 218 ] on which the court suspended his prison sentence until the final appeal hearing in july . [ 219 ] [ 220 ] [ 221 ] [ 222 ] his driver ashok singh , who had given the testimony that it was himself who was driving the car at the time of accident , was charged with perjury for misguiding the court with false testimony and was arrested . the prime witness , police constable ravindra patil went missing multiple times and eventually died in a hospital due to tuberculosis . [ 223 ] in december 2015 , khan was acquitted of all charges from this case due to lack of evidence . [ 224 ] [ 225 ] the supreme court on tuesday 5 july 2016 admitted a plea by the maharashtra government that challenged the acquittal of salman khan in this hit - and - run case by the bombay high court . the court refused to fast - track the case . [ 226 ] 
 relationship with aishwarya rai 
 his relationship with actress aishwarya rai was a well publicised topic in the indian media . [ 227 ] after their break - up in march 2002 , rai accused him of harassing her . she claimed that khan had not been able to come to terms with their break - up and was hounding her ; her parents lodged a complaint against him . [ 228 ] in 2003 , vivek oberoi , rai 's then boyfriend , claimed that khan had threatened him . [ 229 ] in 2005 , news outlets released what was said to be an illicit copy of a mobile phone call recorded in 2001 by the mumbai police . it appeared to be a call in which he threatened rai , in an effort to force her to appear at social events held by mumbai crime figures . the call featured boasts of connections to organised crime and derogatory comments about other actors . however , the alleged tape was tested in the government 's forensic lab in chandigarh , which concluded that it was fake . [ 230 ] [ 231 ] 
 blackbuck hunting and arms act violations cases 
 main article : 1998 blackbuck poaching case 
 the 1998 blackbuck poaching case involves the alleged killing of a deer when khan and his co - stars – saif ali khan , sonali bendre , neelam and tabu – were filming the movie hum saath - saath hain in the forests near jodhpur in 1998 . [ 232 ] [ 233 ] in 2007 , salman spent a week in jodhpur jail before the court granted him bail . in addition to the accusations of poaching of endangered antelopes under the wildlife ( protection ) act , a case under sections 3/25 and 3/27 of the arms act was filed against khan for allegedly keeping and using firearms with an expired licence . [ 234 ] 
 on 17 february 2006 , khan was sentenced to one year in prison for hunting chinkara , an endangered species . the sentence was stayed by a higher court on appeal . [ 235 ] 
 on 10 april 2006 , khan was handed a five - year jail term and remanded to jodhpur jail until 13 april when he was granted bail . [ 236 ] on 24 july 2012 , rajasthan high court finalised charges against khan and his other colleagues in the endangered blackbuck killing case , paving way for the start of the trial . [ 237 ] on 9 july 2014 , the supreme court issued a notice to khan on rajasthan government 's plea challenging the high court order suspending his conviction . [ 238 ] khan was acquitted in the blackbuck and chinkara poaching cases by the rajasthan high court on 24 july 2016 . [ 239 ] 
 on 18 october 2016 the rajasthan government decided to approach the supreme court to challenge khan 's acquittal in two related cases . [ 240 ] 
 on 18 january 2017 khan was acquitted by a jodhpur court in an arms act case linked to the killing of a blackbuck in rajasthan . khan pleaded " not guilty " to charges of violating the law by keeping unlicensed weapons and using them . acquitting the actor , the court said that the prosecution failed to submit enough evidence against him . [ 234 ] [ 241 ] 
 on 5 april 2018 , jodhpur court convicted khan in the blackbuck poaching case and sentenced him to five years imprisonment while acquitting saif ali khan , sonali bendre , neelam , and tabu . [ 242 ] [ 16 ] [ 17 ] 
 khan was released on bail on 7 april 2018 , pending appeal . [ 243 ] [ 244 ] [ 245 ] 
 on 14 april 2024 , two members of the bishnoi gang , who belong a religious sect that considers the blackbuck sacred , opened fire on khan 's apartment in bandra , mumbai before fleeing on a motorcycle . khan , who was inside along with several relatives , was unharmed . the gunmen were later arrested in gujarat . the gang had previously threatened khan over the hunting incident . police said that the attack was masterminded by am nol bishnoi , the younger brother of the gang 's leader lawrence bishnoi . [ 246 ] [ 247 ] 
 remarks about 26/11 attacks 
 in september 2010 , khan was reported to have claimed during an interview on a pakistani channel that the 26/11 attacks got a lot of attention because the " elite " were targeted . [ 248 ] [ 249 ] during the interview the actor had said : " it was the elite that was targeted this time . five star hotels and stuff . so they panicked . then they got up and spoke about it . my question is " why not before ? " attacks have happened in trains and small towns too , but no one talked about it so much . " [ 248 ] [ 249 ] khan also said that pakistan was not to be blamed for it , and that the indian security had failed . khan 's comments had drawn strong reactions from deputy chief minister of maharashtra chagan bhujbal , shiv sena , bjp and other political parties . [ 248 ] the comments were also condemned by ujwal nikam , special prosecutor in the 26/11 trial . [ 248 ] khan later apologised for his comments . [ 248 ] 
 tweets on yakub memon 
 on 25 july 2015 , khan made a number of tweets from his twitter account in support of the 1993 bombay bombings , accused yakub memon . memon was scheduled to be executed , before which khan made his tweets . [ 250 ] khan said that yakub 's brother , tiger memon should be hanged instead . he also asked the prime minister of pakistan , nawaz sharif to confirm " whether the mob boss is in his country . " [ 251 ] the tweets sparked protests outside his house where police had to be deployed and were criticised in the social media and by his father , salim khan . following these incidents , khan withdrew his tweets and apologised . [ 252 ] 
 interference in jiah khan suicide case 
 following the death of sushant singh rajput by hanging , the late actress jiah khan 's mother , rabia amin , revealed that salman khan had attempted to influence the case , where accusations for abetment of suicide were charged against actor sooraj pancholi . a cbi officer who was investigating the case had told rabia amin that salman khan called him every day , asking him not to harass or touch sooraj pancholi , as a lot of money was invested in him . [ 253 ] [ 254 ] 
 threats from lawrence bishnoi gang 
 since 2022 , khan has been the target of repeated threats from the lawrence bishnoi gang , which has claimed hostility towards the actor due to his alleged involvement in the 1998 blackbuck poaching case , as the bishnoi community regards the blackbuck as sacred . [ 255 ] [ 256 ] 
 on 14 april 2024 , two men opened fire outside khan ’s residence , galaxy apartments , in bandra , mumbai . khan , who was at home with his family , was unharmed . the shooters , later arrested in gujarat , were alleged associates of am nol bishnoi , the brother of gang leader lawrence bishnoi . [ 257 ] [ 258 ] 
 in september 2024 , salman khan addressed the threats publicly , remarking that life and death were in god ’s hands and that security measures made his daily routine difficult . [ 259 ] 
 in march 2025 , a bishnoi gang member warned that not only khan but also anyone collaborating with him — actors , producers , or directors — would be targeted . the group also claimed responsibility for two attacks on comedian kapil sharma ’s café in surrey , canada , alleging that he had invited khan to his show . [ 260 ] [ 261 ] 
 discography 
 khan has sung the following hindi songs for bollywood films : 
 list of salman khan film singing credits 
 year title song notes ref . 
 1999 hello brother " cha and i ke daal par " duet with alka yagnik [ 262 ] 
 2014 kick " hai yehi zindagi " – version 2 [ 263 ] 
 " hangover " duet with shreya ghoshal [ 263 ] 
 " jumme ki raat hai " – version 2 duet with palak muchhal [ 263 ] 
 " tu hi tu " – version 2 solo [ 263 ] 
 2015 hero " main hoon hero tera " solo [ 264 ] 
 2016 sultan " baby ko bass pasand hai " solo [ 265 ] 
 " jag ghoomeya " solo [ 266 ] 
 " 440 volt " [ 267 ] 
 " sultan " [ 268 ] 
 2018 race 3 " i found love " [ 269 ] 
 2019 notebook " main taare " solo [ 270 ] 
 dabangg 3 " yu karke " duet with payal dev [ 271 ] 
 2020 pyar karona " pyar karona " written along with hussain dalal [ 272 ] 
 tere bina " tere bina " co - starring jacqueline fernandez [ 273 ] 
 bhai bhai " bhai bhai " duet with ruhaan arshad and co - written by danish sabri [ 274 ] [ 275 ] 
 accolades 
 main article : list of awards and nominations received by salman khan 
 khan won numerous awards including two national film awards and a filmfare award . 
 notes 
 ^ khan 's full legal name has been reported and recorded as both " abdul rashid salim salman khan " [ 1 ] and " salman salim khan " . [ 2 ] 
 references 
 ^ 
 " sunny deol , salman khan , dilip kumar , sridevi : bollywood celebs and their surprising original names " . the times of india . 6 november 2024 . 
 ^ 
 " salman salim khan | zaubacorp " . www . zaubacorp . com . retrieved 30 july 2025 . 
 ^ 
 " salman khan on dancing in awards shows " . the indian express . 17 july 2017 . 
 ^ 
 respers france , lisa ( 6 may 2015 ) . " who is salman khan ? only one of the world 's biggest stars " . cnn . retrieved 30 january 2023 . 
 ^ 
 " salman khan : hail bollywood 's new king " . hindustan times . 28 august 2012 . archived from the original on 26 august 2013 . retrieved 17 january 2013 . 
 ^ 
 robehmed , natalie . " salman khan 's earnings : $ 33 . 5 million in 2015 " . forbes . retrieved 14 may 2019 . 
 ^ 
 a 
 b 
 " salman khan makes it to forbes rich list , shah rukh khan doesn ' t " . ndtv . pti . 30 june 2015 . retrieved 16 october 2023 . 
 ^ 
 " top 100 highest paid celebrity entertainers of world 2018 " . forbes . retrieved 1 november 2018 . 
 ^ 
 " box office special features – highest grosser of the year " . bollywood hungama . 29 december 2020 . retrieved 3 august 2021 . 
 ^ 
 pathak , ankur ( 7 january 2014 ) . " salman khan does it again " . the times of india . retrieved 27 december 2019 . 
 ^ 
 " fourth season of bigg boss " . emirates 24 / 7 . 1 september 2014 . archived from the original on 7 september 2014 . retrieved 22 september 2014 . 
 ^ 
 " salman khan : bollywood star jailed for five years in hit - and - run case " . bbc news . 6 may 2015 . retrieved 6 may 2015 . 
 ^ 
 " salman khan hit - and - run case : other high - profile hit - and - run cases " . india today . 6 may 2015 . retrieved 6 may 2015 . 
 ^ 
 " as it happened : salman khan sentenced to 5 years in jail in hit - and - run case " . the hindu . 6 may 2015 . retrieved 6 may 2015 . 
 ^ 
 " hit - and - run : salman walks free after lower court order quashed " . the indian express . 11 december 2015 . 
 ^ 
 a 
 b 
 " blackbuck poaching case : salman khan gets 5-year jail term " . the economic times . 5 april 2018 . retrieved 5 april 2018 . 
 ^ 
 a 
 b 
 " bollywood star khan jailed for five years " . bbc . 5 april

---
### 8. [System Design Interview - YouTube](https://m.youtube.com/c/systemdesigninterview)
*m.youtube.com*  
**🌐 URL:** [https://m.youtube.com/c/systemdesigninterview](https://m.youtube.com/c/systemdesigninterview)

**Snippet**
> system design interview 
 @ system design interview 87 . 2 k subscribers 7 videos 
 preparing for a system design interview ? or simply want to know how distributed systems work ? welcome to our chann...

**Summary**
> system design interview 
 @ system design interview 87 . 2 k subscribers 7 videos 
 preparing for a system design interview ? or simply want to know how distributed systems work ? welcome to our channel ! 
 system design interview channel introduction 
 videos 
 system design interview – step by step guide 
 system design interview - top k problem ( heavy hitters ) 
 system design interview - distributed cache

---
### 9. [System Design Series' Articles | daily.dev](https://app.daily.dev/posts/GSgJ_l3DL)
*app.daily.dev*  
**🌐 URL:** [https://app.daily.dev/posts/GSgJ_l3DL](https://app.daily.dev/posts/GSgJ_l3DL)

**Snippet**
> our lawyers advised us to tell you that we use cookies 
 to improve user experience . 
 i like cookies

**Summary**
> our lawyers advised us to tell you that we use cookies 
 to improve user experience . 
 i like cookies

---
### 10. [System Design: REST, GraphQL, gRPC | by Karan Pratap Singh | Medium](https://medium.com/@karan99/system-design-rest-graphql-grpc-5f12c16f3f09?responsesOpen=true&sortBy=REVERSE_CHRON)
*medium.com*  
**🌐 URL:** [https://medium.com/@karan99/system-design-rest-graphql-grpc-5f12c16f3f09?responsesOpen=true&sortBy=REVERSE_CHRON](https://medium.com/@karan99/system-design-rest-graphql-grpc-5f12c16f3f09?responsesOpen=true&sortBy=REVERSE_CHRON)

**Snippet**
> a good api design is always a crucial part of any system . but it is also important to pick the right api technology . so , in this tutorial , we will briefly discuss different api technologies such a...

**Summary**
> a good api design is always a crucial part of any system . but it is also important to pick the right api technology . so , in this tutorial , we will briefly discuss different api technologies such as rest , graphql , and grpc . 
 what ’s an api ? 
 before we even get into api technologies , let ’s first understand what is an api . 
 an api is a set of definitions and protocols for building and integrating application software . it ’s sometimes referred to as a contract between an information provider and an information user establishing the content required from the producer and the content required by the consumer . 
 in other words , if you want to interact with a computer or system to retrieve information or perform a function , an api helps you communicate what you want to that system so it can understand and complete the request . 
 rest 
 a rest api ( also known as restful api ) is an application programming interface that conforms to the constraints of rest architectural style and allows for interaction with restful web services . rest stands for representational state transfer and it was first introduced by roy fielding in the year 2000 . 
 in rest api , the fundamental unit is a resource . 
 concepts 
 let ’s discuss some concepts of a restful api . 
 constraints 
 in order for an api to be considered restful , it has to conform to these architectural constraints : 
 uniform interface : there should be a uniform way of interacting with a given server . 
 client - server : a client - server architecture managed through http . 
 stateless : no client context shall be stored on the server between requests . 
 cacheable : every response should include whether the response is cacheable or not and for how much duration responses can be cached at the client - side . 
 layered system : an application architecture needs to be composed of multiple layers . 
 code on demand : return executable code to support a part of your application . ( optional ) 
 http verbs 
 http defines a set of request methods to indicate the desired action to be performed for a given resource . although they can also be nouns , these request methods are sometimes referred to as http verbs . each of them implements a different semantic , but some common features are shared by a group of them . 
 below are some commonly used http verbs : 
 get : request a representation of the specified resource . 
 head : response is identical to a request , but without the response body . 
 post : submits an entity to the specified resource , often causing a change in state or side effects on the server . 
 put : replaces all current representations of the target resource with the request payload . 
 delete : deletes the specified resource . 
 patch : applies partial modifications to a resource . 
 http response codes 
 http response status codes indicate whether a specific http request has been successfully completed . 
 there are five classes defined by the standard : 
 1xx — informational responses . 
 2xx — successful responses . 
 3xx — redirection responses . 
 4xx — client error responses . 
 5xx — server error responses . 
 for example , http 200 means that the request was successful . 
 advantages 
 let ’s discuss some advantages of rest api : 
 simple and easy to understand . 
 flexible and portable . 
 good caching support . 
 client and server are decoupled . 
 disadvantages 
 let ’s discuss some disadvantages of rest api : 
 over - fetching of data . 
 sometimes multiple round trips to the server are required . 
 use cases 
 rest apis are pretty much used universally and are the default standard for designing apis . overall rest apis are quite flexible and can fit almost all scenarios . 
 example 
 here ’s an example usage of a rest api that operates on a users resource . 
 | uri | http verb | description | | — — — — — — — | — — — — — | — — — — — — — — — — | | /users | get | get all users | | / users / { id } | get | get a user by id | | /users | post | add a new user | | / users / { id } | patch | update a user by id | | / users / { id } | delete | delete a user by id | 
 there is so much more to learn when it comes to rest apis , i will highly recommend looking into hypermedia as the engine of application state ( hateoas ) . 
 graphql 
 graphql is a query language and server - side runtime for apis that prioritizes giving clients exactly the data they request and no more . it was developed by facebook and later open - sourced in 2015 . 
 graphql is designed to make apis fast , flexible , and developer - friendly . additionally , graphql gives api maintainers the flexibility to add or deprecate fields without impacting existing queries . developers can build apis with whatever methods they prefer , and the graphql specification will ensure they function in predictable ways to clients . 
 in graphql , the fundamental unit is a query . 
 concepts 
 let ’s briefly discuss some key concepts in graphql : 
 schema 
 a graphql schema describes the functionality clients can utilize once they connect to the graphql server . 
 queries 
 a query is a request made by the client . it can consist of fields and arguments for the query . the operation type of a query can also be a mutation which provides a way to modify server - side data . 
 resolvers 
 resolver is a collection of functions that generate responses for a graphql query . in simple terms , a resolver acts as a graphql query handler . 
 advantages 
 let ’s discuss some advantages of graphql : 
 eliminates over - fetching of data . 
 strongly defined schema . 
 code generation support . 
 payload optimization . 
 disadvantages 
 let ’s discuss some disadvantages of graphql : 
 shifts complexity to server - side . 
 caching becomes hard . 
 versioning is ambiguous . 
 n+1 problem . 
 use cases 
 graphql proves to be essential in the following scenarios : 
 reducing app bandwidth usage as we can query multiple resources in a single query . 
 rapid prototyping for complex systems . 
 when we are working with a graph - like data model . 
 example 
 here ’s a graphql schema that defines a type and a type . 
 using the above schema , the client can request the required fields easily without having to fetch the entire resource or guess what the api might return . 
 this will give the following response to the client . 
 learn more about graphql at graphql . org . 
 grpc 
 grpc is a modern open - source high - performance remote procedure call ( rpc ) framework that can run in any environment . it can efficiently connect services in and across data centers with pluggable support for load balancing , tracing , health checking , authentication and much more . 
 concepts 
 let ’s discuss some key concepts of grpc . 
 protocol buffers 
 protocol buffers provide a language and platform - neutral extensible mechanism for serializing structured data in a forward and backward - compatible way . it ’s like json , except it ’s smaller and faster , and it generates native language bindings . 
 service definition 
 like many rpc systems , grpc is based on the idea of defining a service and specifying the methods that can be called remotely with their parameters and return types . grpc uses protocol buffers as the interface definition language ( idl ) for describing both the service interface and the structure of the payload messages . 
 advantages 
 let ’s discuss some disadvantages of grpc : 
 lightweight and efficient . 
 high performance . 
 built - in code generation support . 
 bi - directional streaming . 
 disadvantages 
 let ’s discuss some disadvantages of grpc : 
 relatively new compared to rest and graphql . 
 limited browser support . 
 steeper learning curve . 
 not human readable . 
 use cases 
 below are some good use cases for grpc : 
 real - time communication via bi - directional streaming . 
 efficient inter - service communication in microservices . 
 low latency and high throughput communication . 
 polyglot environments . 
 example 
 here ’s a basic example of a grpc service defined in a file . using this definition , we can easily code generate the service in the programming language of our choice . 
 rest vs graphql vs grpc 
 now that we know how these api designing techniques work , let ’s compare them based on the following parameters : 
 will it cause tight coupling ? 
 how chatty ( distinct api calls to get needed information ) are the apis ? 
 what ’s the performance like ? 
 how complex is it to integrate ? 
 how well does the caching work ? 
 built - in tooling and code generation ? 
 what ’s api discoverability like ? 
 how easy is it to version apis ? 
 | type | coupling | chattiness | performance | complexity | caching | codegen | discoverability | versioning | | — — — — | — — — — | — — — — — | — — — — — — | — — — — — | — — — — | — — — — | — — — — — — — — | — — — — — | | rest | low | high | good | medium | great | bad | good | easy | | graphql | medium | low | good | high | custom | good | good | custom | | grpc | high | medium | great | low | custom | great | bad | hard | 
 which api technology is better ? 
 well , the answer is none of them . there is no silver bullet as each of these technologies has its own advantages and disadvantages . users only care about using our apis in a consistent way , so make sure to focus on your domain and requirements when designing your api . 
 this article is part of my open - source system design course available on github . 
 distributed systems 
 system design interview 
 architecture 
 software engineering


In [9]:
import requests
from typing import List, Optional, Dict, Any
from dataclasses import dataclass
from urllib.parse import urljoin


@dataclass
class FirecrawlResponse:
    success: bool
    data: Optional[Dict[str, Any]] = None
    error: Optional[str] = None
    status_code: Optional[int] = None


class FirecrawlClient:
    """
    Production-grade client for interacting with Firecrawl API.
    Provides methods for scraping, crawling, extracting, and verifying service health.
    """

    def __init__(self, base_url: str = "http://localhost:8010", timeout: int = 30):
        self.base_url = base_url.rstrip("/")
        self.timeout = timeout

    def _handle_response(self, response: requests.Response) -> FirecrawlResponse:
        try:
            json_data = response.json()
        except requests.JSONDecodeError:
            return FirecrawlResponse(
                success=False,
                error="Invalid JSON response",
                status_code=response.status_code,
            )

        if response.ok:
            return FirecrawlResponse(success=True, data=json_data, status_code=response.status_code)
        else:
            return FirecrawlResponse(success=False, error=json_data, status_code=response.status_code)

    def _post(self, endpoint: str, payload: Dict[str, Any]) -> FirecrawlResponse:
        url = urljoin(self.base_url + "/", endpoint.lstrip("/"))
        try:
            r = requests.post(url, json=payload, timeout=self.timeout)
            return self._handle_response(r)
        except requests.RequestException as e:
            return FirecrawlResponse(success=False, error=str(e))

    def _get(self, endpoint: str) -> FirecrawlResponse:
        url = urljoin(self.base_url + "/", endpoint.lstrip("/"))
        try:
            r = requests.get(url, timeout=self.timeout)
            return self._handle_response(r)
        except requests.RequestException as e:
            return FirecrawlResponse(success=False, error=str(e))

    # --- API Methods ---

    def health(self) -> FirecrawlResponse:
        """Check server health"""
        return self._get("/health")

    def scrape(self, url: str) -> FirecrawlResponse:
        """Scrape a single URL"""
        return self._post("/scrape", {"url": url})

    def crawl(self, url: str) -> FirecrawlResponse:
        """Start a crawl job"""
        return self._post("/crawl", {"url": url})

    def crawl_status(self, crawl_id: str) -> FirecrawlResponse:
        """Fetch crawl job status"""
        return self._get(f"/crawl/{crawl_id}")

    def extract(self, urls: List[str]) -> FirecrawlResponse:
        """Start extraction job for one or more URLs"""
        return self._post("/extract", {"urls": urls})

    def root(self) -> FirecrawlResponse:
        """Fetch API root details"""
        return self._get("/")


In [11]:
if __name__ == "__main__":
    client = FirecrawlClient()

    print("Health:", client.health())
    print("Scrape:", client.scrape("https://medium.com/@karan99/system-design-rest-graphql-grpc-5f12c16f3f09"))
    crawl_job = client.crawl("https://medium.com/@karan99/system-design-rest-graphql-grpc-5f12c16f3f09")
    if crawl_job.success and crawl_job.data and "data" in crawl_job.data and "id" in crawl_job.data["data"]:
        crawl_id = crawl_job.data["data"]["id"]
        print("Crawl Job:", crawl_id)
        print("Crawl Status:", client.crawl_status(crawl_id))

    extract_job = client.extract(["https://medium.com/@karan99/system-design-rest-graphql-grpc-5f12c16f3f09"])
    print("Extract:", extract_job)

Health: FirecrawlResponse(success=True, data={'status': 'healthy', 'firecrawl_status': 'up', 'timestamp': '2025-10-16T21:58:34.934124', 'firecrawl_url': 'http://firecrawl-api-1:3002'}, error=None, status_code=200)
Scrape: FirecrawlResponse(success=True, data={'success': True, 'data': {'success': True, 'data': {'markdown': '[Sitemap](https://medium.com/sitemap/sitemap.xml)\n\n[Open in app](https://rsci.app.link/?%24canonical_url=https%3A%2F%2Fmedium.com%2Fp%2F5f12c16f3f09&%7Efeature=LoOpenInAppButton&%7Echannel=ShowPostUnderUser&%7Estage=mobileNavBar&source=post_page---top_nav_layout_nav-----------------------------------------)\n\nSign up\n\n[Sign in](https://medium.com/m/signin?operation=login&redirect=https%3A%2F%2Fmedium.com%2F%40karan99%2Fsystem-design-rest-graphql-grpc-5f12c16f3f09&source=post_page---top_nav_layout_nav-----------------------global_nav------------------)\n\n[Medium Logo](https://medium.com/?source=post_page---top_nav_layout_nav--------------------------------------